In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
def rolling_mean(series):
    return series.fillna(0).rolling(window=3).mean().mean()

def rolling_max(series):
    return series.fillna(0).rolling(window=3).mean().max()

def rolling_min(series):
    return series.fillna(0).rolling(window=3).mean().min()

def rolling_std(series):
    return series.fillna(0).rolling(window=3).mean().std()

def diffmean(s):
    return s.dropna().drop_duplicates().diff().mean()

In [3]:
items = pd.read_csv("cleaned_data/items.csv")
items_categories = pd.read_csv("cleaned_data/item_categories.csv")
sales_train = pd.read_csv("cleaned_data/sales_train.csv")
shops = pd.read_csv("cleaned_data/shops.csv")

df_init = pd.merge(items, items_categories, on="item_category_id", how="inner")
df_init = pd.merge(df_init, sales_train, on="item_id", how="inner")
df_init = pd.merge(df_init, shops, on="shop_id", how="inner")
df_init.head()

,item_name,item_id,item_category_id,item_category_name,item_category_type,item_category_subtype,date,date_block_num,shop_id,item_price,item_cnt_day,shop_name,shop_city
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино,DVD,01.09.2014,20,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,24.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,12.11.2014,22,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,05.07.2014,18,54,100.0,1.0,"Химки ТЦ ""Мега""",Химки
4,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,26.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки


## DELETE OUTLIERS

### Take 99th percentile of sells in a day

In [4]:
'''print(len(df_init))
df_init = df_init[df_init["item_cnt_day"] <= np.percentile(a=df_init["item_cnt_day"], q=99)]
print(len(df_init))
df_init.head()'''

'print(len(df_init))\ndf_init = df_init[df_init["item_cnt_day"] <= np.percentile(a=df_init["item_cnt_day"], q=99)]\nprint(len(df_init))\ndf_init.head()'

### Delete negative item_cnt_day -> Devolutions

In [5]:
'''(df_init["item_cnt_day"] < 0).sum()'''

'(df_init["item_cnt_day"] < 0).sum()'

In [6]:
'''print(len(df_init))
df_init = df_init[df_init["item_cnt_day"] > 0]
print(len(df_init))
df_init.head()'''

'print(len(df_init))\ndf_init = df_init[df_init["item_cnt_day"] > 0]\nprint(len(df_init))\ndf_init.head()'

In [ ]:
df_ts["item_cnt_day"].clip(0, 20, inplace=True)


## DEFINE FEATURES PARAMETERS

In [8]:
month_to_predict = 34
max_train_month = 33
features_folder_name = "features_prediction/"

In [9]:
#save df aux if we need all months
df_aux = df_init.copy()
df = df_init[df_init["date_block_num"] < month_to_predict]
assert(df["date_block_num"].max() == max_train_month)
print("Maximum month in features after processing will be:", df.date_block_num.value_counts().index.max())
df.head()

Maximum month in features after processing will be: 33


,item_name,item_id,item_category_id,item_category_name,item_category_type,item_category_subtype,date,date_block_num,shop_id,item_price,item_cnt_day,shop_name,shop_city
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино,DVD,01.09.2014,20,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,24.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,12.11.2014,22,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,05.07.2014,18,54,100.0,1.0,"Химки ТЦ ""Мега""",Химки
4,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,26.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки


# DELETE UNNECESARY COLUMNS

In [10]:
df.drop(columns=["item_name", "item_category_name", "date", "shop_name"], inplace=True)
df.head()

,item_id,item_category_id,item_category_type,item_category_subtype,date_block_num,shop_id,item_price,item_cnt_day,shop_city
0,0,40,Кино,DVD,20,54,58.0,1.0,Химки
1,2,40,Кино,DVD,19,54,58.0,1.0,Химки
2,2,40,Кино,DVD,22,54,58.0,1.0,Химки
3,3,40,Кино,DVD,18,54,100.0,1.0,Химки
4,3,40,Кино,DVD,19,54,58.0,1.0,Химки


# DATAFRAME TO CALCULATE TIME SERIES FEATURES

In [11]:
df_ts = df.groupby(["item_id", "shop_id", "date_block_num"]).agg({"item_cnt_day":"sum"}).reset_index().rename(columns={"item_cnt_day":"item_cnt_month"})
df_ts.head()

,item_id,shop_id,date_block_num,item_cnt_month
0,0,54,20,1.0
1,1,55,15,2.0
2,1,55,18,1.0
3,1,55,19,1.0
4,1,55,20,1.0


In [12]:
all_months = [x for x in range(0, month_to_predict + 1)]

In [13]:
combinations = []
for month in all_months:
    all_items = list(sales_train[sales_train.date_block_num == month].item_id.unique())
    all_shops = list(sales_train[sales_train.date_block_num == month].shop_id.unique())
    
    if month == 34:
        all_items = pd.read_csv("cleaned_data/test.csv")["item_id"].unique()
        all_shops = pd.read_csv("cleaned_data/test.csv")["shop_id"].unique()
    
    for item_id in all_items:
        for shop_id in all_shops:
            combinations.append([item_id, shop_id, month])

combinations = pd.DataFrame(combinations, columns=["item_id", "shop_id", "date_block_num"])
combinations.head()

,item_id,shop_id,date_block_num
0,22154,59,0
1,22154,25,0
2,22154,24,0
3,22154,23,0
4,22154,19,0


In [14]:
df_ts = pd.merge(combinations, df_ts, how="left", on=["item_id", "shop_id", "date_block_num"])
df_ts["item_cnt_month"].fillna(0, inplace=True)
df_ts = pd.merge(df_ts, items, on="item_id", how="inner").drop(columns=["item_name"])
df_ts = pd.merge(df_ts, items_categories, on="item_category_id", how="inner")
df_ts = pd.merge(df_ts, shops, on="shop_id", how="inner").drop(columns=["shop_name"])
df_ts.drop("item_category_name", axis=1, inplace=True)
df_ts["item_cnt_month"].clip(0, 20, inplace=True)
df_ts.sample(10)

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,item_category_type,item_category_subtype,shop_city
8407170,13697,51,25,1.0,69,Подарки,Сувениры,Тюмень
4453372,10578,16,30,0.0,57,Музыка,MP3,Коломна
5284885,17259,13,13,0.0,37,Кино,Blu,Казань
3874076,2625,57,21,0.0,55,Музыка,CD локального производства,Якутск
852931,17811,19,32,1.0,40,Кино,DVD,Курск
9090767,15104,44,9,0.0,62,Подарки,"Гаджеты, роботы, спорт",Самара
649816,9281,24,4,0.0,62,Подарки,"Гаджеты, роботы, спорт",Москва
5505912,11811,12,12,3.0,19,Игры,PS3,Интернет-магазин
1604747,16443,21,13,0.0,40,Кино,DVD,Москва
2053323,8319,27,1,0.0,40,Кино,DVD,Москва


In [15]:
df_aspects = df_ts[df_ts.date_block_num == month_to_predict]
print(len(df_aspects))
df_ts = df_ts[df_ts.date_block_num <= max_train_month]
print(len(df_ts))

214200
10913850


In [16]:
max_train_month

33

In [17]:
print(df_ts.date_block_num.max())
print(df_aspects["date_block_num"].value_counts())

33
Series([], Name: date_block_num, dtype: int64)


## FEATURES

### Of categories and shops (shop_id & item_category_id)

In [18]:
categories_and_shops = df_aspects[["shop_id", "item_category_id"]].drop_duplicates(subset=["shop_id", "item_category_id"])
categories_and_shops.head()

,shop_id,item_category_id
29,5,19
4229,5,55
24389,5,23
28829,5,20
32939,5,30


#### Items sold from the same categorie in the same shop historically by month (mean, sum, std, min, max)

In [19]:
items_same_cat_and_shop = df_ts.groupby(["shop_id", "item_category_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()

In [20]:
items_same_cat_and_shop.head()

,shop_id,item_category_id,date_block_num,item_cnt_month
0,2,0,0,0.0
1,2,0,1,0.0
2,2,0,2,0.0
3,2,0,3,0.0
4,2,0,4,0.0


In [21]:
items_same_cat_and_shop_h = items_same_cat_and_shop.groupby(["shop_id", "item_category_id"]).agg({"item_cnt_month":["sum", "mean", "std", "min", "max"]}).reset_index()

items_same_cat_and_shop_h.columns = ['_'.join(col).strip() for col in items_same_cat_and_shop_h.columns.values]

items_same_cat_and_shop_h.rename(columns={
   "shop_id_":"shop_id",
    "item_category_id_":"item_category_id",
    "item_cnt_month_sum":"hist_sales_sum_cat_by_shop",
    "item_cnt_month_mean":"hist_sales_mean_cat_by_shop",
    "item_cnt_month_std":"hist_sales_std_cat_by_shop",
    "item_cnt_month_min":"hist_sales_min_cat_by_shop",
    "item_cnt_month_max":"hist_sales_max_cat_by_shop"
}, inplace=True)

print(len(categories_and_shops))
categories_and_shops = pd.merge(categories_and_shops, items_same_cat_and_shop_h, on=["shop_id", "item_category_id"], how="left")
print(len(categories_and_shops))
categories_and_shops.head()

2604
2604


,shop_id,item_category_id,hist_sales_sum_cat_by_shop,hist_sales_mean_cat_by_shop,hist_sales_std_cat_by_shop,hist_sales_min_cat_by_shop,hist_sales_max_cat_by_shop
0,5,19,1401.0,48.310345,26.338597,0.0,118.0
1,5,55,2725.0,93.965517,36.425543,0.0,167.0
2,5,23,1045.0,36.034483,20.169784,0.0,80.0
3,5,20,953.0,32.862069,31.413969,0.0,113.0
4,5,30,3467.0,119.551724,40.653225,0.0,173.0


#### Number of item sold of category in shop last n months

In [22]:
months = [max_train_month - i for i in range(3)]
months

[28, 27, 26]

In [23]:
print(len(categories_and_shops))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby(["item_category_id", "shop_id"]).agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "item_category_id_":"item_category_id",
        "shop_id_":"shop_id",
        "item_cnt_month_sum":"item_category_and_shop_sum_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"item_category_and_shop_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"item_category_and_shop_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"item_category_and_shop_std_shifted_" + str(month_to_predict - month) + "_months"        
    },
    inplace=True)
    categories_and_shops = pd.merge(categories_and_shops, items_sold_in_month, on=["item_category_id", "shop_id"], how="left")
print(len(categories_and_shops))
categories_and_shops.head()

2604
2604


,shop_id,item_category_id,hist_sales_sum_cat_by_shop,hist_sales_mean_cat_by_shop,hist_sales_std_cat_by_shop,hist_sales_min_cat_by_shop,hist_sales_max_cat_by_shop,item_category_and_shop_mean_shifted_1_months,item_category_and_shop_sum_shifted_1_months,item_category_and_shop_max_shifted_1_months,item_category_and_shop_std_shifted_1_months,item_category_and_shop_mean_shifted_2_months,item_category_and_shop_sum_shifted_2_months,item_category_and_shop_max_shifted_2_months,item_category_and_shop_std_shifted_2_months,item_category_and_shop_mean_shifted_3_months,item_category_and_shop_sum_shifted_3_months,item_category_and_shop_max_shifted_3_months,item_category_and_shop_std_shifted_3_months
0,5,19,1401.0,48.310345,26.338597,0.0,118.0,0.500000,70.0,9.0,1.232066,0.278571,39.0,2.0,0.576118,0.335714,47.0,7.0,0.894513
1,5,55,2725.0,93.965517,36.425543,0.0,167.0,0.144345,97.0,3.0,0.431610,0.120536,81.0,6.0,0.445605,0.186012,125.0,4.0,0.490974
2,5,23,1045.0,36.034483,20.169784,0.0,80.0,0.290541,43.0,4.0,0.740044,0.128378,19.0,4.0,0.484879,0.263514,39.0,4.0,0.712965
3,5,20,953.0,32.862069,31.413969,0.0,113.0,0.452555,62.0,20.0,1.890265,0.262774,36.0,15.0,1.362686,0.408759,56.0,16.0,1.691478
4,5,30,3467.0,119.551724,40.653225,0.0,173.0,0.897059,122.0,14.0,1.790011,1.088235,148.0,20.0,2.714762,0.867647,118.0,7.0,1.533859


In [24]:
categories_and_shops.to_csv("generated/" + features_folder_name + "features_shop_and_categories.csv", index=False)

### Of categories types and shops (shop_id & item_category_type)

In [25]:
category_type_and_shops = df_aspects[["shop_id", "item_category_type"]].drop_duplicates(subset=["shop_id", "item_category_type"])
category_type_and_shops.head()

,shop_id,item_category_type
29,5,Игры
4229,5,Музыка
37769,5,Подарки
43529,5,Карты
43649,5,Аксессуары


#### Historically stats

In [26]:
items_same_cat_type_and_shop = df_ts.groupby(["shop_id", "item_category_type", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()

In [27]:
items_same_cat_type_and_shop_h = items_same_cat_type_and_shop.groupby(["shop_id", "item_category_type"]).agg({"item_cnt_month":["sum", "mean", "std", "min", "max"]}).reset_index()

items_same_cat_type_and_shop_h.columns = ['_'.join(col).strip() for col in items_same_cat_type_and_shop_h.columns.values]

items_same_cat_type_and_shop_h.rename(columns={
   "shop_id_":"shop_id",
    "item_category_type_":"item_category_type",
    "item_cnt_month_sum":"hist_sales_sum_cat_type_by_shop",
    "item_cnt_month_mean":"hist_sales_mean_cat_type_by_shop",
    "item_cnt_month_std":"hist_sales_std_cat_type_by_shop",
    "item_cnt_month_min":"hist_sales_min_cat_type_by_shop",
    "item_cnt_month_max":"hist_sales_max_cat_type_by_shop"
}, inplace=True)

print(len(category_type_and_shops))
category_type_and_shops = pd.merge(category_type_and_shops, items_same_cat_type_and_shop_h, on=["shop_id", "item_category_type"], how="left")
print(len(category_type_and_shops))
category_type_and_shops.head()

546
546


,shop_id,item_category_type,hist_sales_sum_cat_type_by_shop,hist_sales_mean_cat_type_by_shop,hist_sales_std_cat_type_by_shop,hist_sales_min_cat_type_by_shop,hist_sales_max_cat_type_by_shop
0,5,Игры,8036.0,277.103448,133.174253,0.0,637.0
1,5,Музыка,2759.0,95.137931,37.232593,0.0,174.0
2,5,Подарки,2795.0,96.379310,65.421389,0.0,303.0
3,5,Карты,369.0,12.724138,7.777700,0.0,28.0
4,5,Аксессуары,532.0,18.344828,14.519140,0.0,58.0


#### Shifted n months

In [28]:
months = [max_train_month - i for i in range(3)]
months

[28, 27, 26]

In [29]:
print(len(category_type_and_shops))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby(["item_category_type", "shop_id"]).agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "item_category_type_":"item_category_type",
        "shop_id_":"shop_id",
        "item_cnt_month_sum":"item_category_type_and_shop_sum_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"item_category_type_and_shop_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"item_category_type_and_shop_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"item_category_type_and_shop_std_shifted_" + str(month_to_predict - month) + "_months"        
    },
    inplace=True)
    category_type_and_shops = pd.merge(category_type_and_shops, items_sold_in_month, on=["item_category_type", "shop_id"], how="left")
print(len(category_type_and_shops))
category_type_and_shops.head()

546
546


,shop_id,item_category_type,hist_sales_sum_cat_type_by_shop,hist_sales_mean_cat_type_by_shop,hist_sales_std_cat_type_by_shop,hist_sales_min_cat_type_by_shop,hist_sales_max_cat_type_by_shop,item_category_type_and_shop_mean_shifted_1_months,item_category_type_and_shop_sum_shifted_1_months,item_category_type_and_shop_max_shifted_1_months,item_category_type_and_shop_std_shifted_1_months,item_category_type_and_shop_mean_shifted_2_months,item_category_type_and_shop_sum_shifted_2_months,item_category_type_and_shop_max_shifted_2_months,item_category_type_and_shop_std_shifted_2_months,item_category_type_and_shop_mean_shifted_3_months,item_category_type_and_shop_sum_shifted_3_months,item_category_type_and_shop_max_shifted_3_months,item_category_type_and_shop_std_shifted_3_months
0,5,Игры,8036.0,277.103448,133.174253,0.0,637.0,0.347866,375.0,20.0,1.288673,0.256957,277.0,20.0,1.184009,0.292208,315.0,16.0,1.000907
1,5,Музыка,2759.0,95.137931,37.232593,0.0,174.0,0.095969,100.0,3.0,0.359318,0.079655,83.0,6.0,0.364607,0.119962,125.0,4.0,0.404115
2,5,Подарки,2795.0,96.379310,65.421389,0.0,303.0,0.183183,183.0,20.0,0.954620,0.150150,150.0,20.0,0.828765,0.195195,195.0,20.0,1.021285
3,5,Карты,369.0,12.724138,7.777700,0.0,28.0,1.538462,20.0,9.0,2.633609,1.076923,14.0,5.0,1.891276,1.230769,16.0,7.0,2.420532
4,5,Аксессуары,532.0,18.344828,14.519140,0.0,58.0,0.266667,20.0,4.0,0.643750,0.146667,11.0,3.0,0.455991,0.146667,11.0,2.0,0.425324


In [30]:
len(df_ts[(df_ts.shop_id == 5) & (df_ts.item_category_type == "Игры") & (df_ts.date_block_num == 32)]["item_cnt_month"])

0

In [31]:
category_type_and_shops[(category_type_and_shops.shop_id == 5) & (category_type_and_shops.item_category_type == "Игры")]

,shop_id,item_category_type,hist_sales_sum_cat_type_by_shop,hist_sales_mean_cat_type_by_shop,hist_sales_std_cat_type_by_shop,hist_sales_min_cat_type_by_shop,hist_sales_max_cat_type_by_shop,item_category_type_and_shop_mean_shifted_1_months,item_category_type_and_shop_sum_shifted_1_months,item_category_type_and_shop_max_shifted_1_months,item_category_type_and_shop_std_shifted_1_months,item_category_type_and_shop_mean_shifted_2_months,item_category_type_and_shop_sum_shifted_2_months,item_category_type_and_shop_max_shifted_2_months,item_category_type_and_shop_std_shifted_2_months,item_category_type_and_shop_mean_shifted_3_months,item_category_type_and_shop_sum_shifted_3_months,item_category_type_and_shop_max_shifted_3_months,item_category_type_and_shop_std_shifted_3_months
0,5,Игры,8036.0,277.103448,133.174253,0.0,637.0,0.347866,375.0,20.0,1.288673,0.256957,277.0,20.0,1.184009,0.292208,315.0,16.0,1.000907


In [32]:
category_type_and_shops.to_csv("generated/" + features_folder_name + "features_shop_and_category_types.csv", index=False)

### Of categories (category_id)

In [33]:
categories_features = df_aspects[["item_category_id"]].drop_duplicates(subset=["item_category_id"])
print(len(categories_features))
categories_features.head()

62


,item_category_id
29,19
4229,55
24389,23
28829,20
32939,30


#### Total shops that sell items with that category

In [34]:
group = df_aux.groupby("item_category_id").agg({"shop_id":"nunique"}).reset_index().rename(columns={"shop_id":"total_shops_of_category"})
print(len(categories_features))
categories_features = pd.merge(categories_features, group, on="item_category_id", how="left")
print(len(categories_features))
categories_features.sample(10)

62
62


,item_category_id,total_shops_of_category
11,24,51
0,19,55
42,58,53
7,35,55
4,30,55
32,64,55
31,79,51
47,57,49
43,75,54
61,44,1


#### Total items by category

In [35]:
items_by_category = df_aux.groupby("item_category_id").agg({"item_id":"nunique"}).reset_index().rename(columns={"item_id":"total_items_of_category"})
items_by_category.sample(7)

,item_category_id,total_items_of_category
9,9,12
41,41,332
17,17,7
11,11,41
63,63,362
50,50,2
62,62,295


In [36]:
print(len(categories_features))
categories_features = pd.merge(categories_features, items_by_category, on="item_category_id", how="left")
print(len(categories_features))
categories_features.sample(7)

62
62


,item_category_id,total_shops_of_category,total_items_of_category
57,36,1,5
11,24,51,116
58,27,1,7
40,73,52,36
46,5,50,29
33,63,56,362
55,78,1,273


#### Historical mean sales of category by month

In [37]:
df_ts.head()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,item_category_type,item_category_subtype,shop_city
0,5037,5,0,0.0,19,Игры,PS3,Вологда
1,5037,5,1,0.0,19,Игры,PS3,Вологда
2,5037,5,2,0.0,19,Игры,PS3,Вологда
3,5037,5,3,0.0,19,Игры,PS3,Вологда
4,5037,5,4,0.0,19,Игры,PS3,Вологда


In [38]:
sum_of_sales_of_category = df_ts.groupby(["item_category_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_sales_of_category = sum_of_sales_of_category.groupby("item_category_id").agg({"item_cnt_month":"mean"}).reset_index().rename(columns={"item_cnt_month":"mean_sales_of_category_by_month"})
print(len(categories_features))
categories_features = pd.merge(categories_features, mean_sales_of_category, on="item_category_id", how="left")
print(len(categories_features))
categories_features.sample(10)

62
62


,item_category_id,total_shops_of_category,total_items_of_category,mean_sales_of_category_by_month
34,47,45,129,6.965517
16,37,55,1777,1452.586207
20,67,56,393,541.586207
12,21,52,125,212.275862
8,2,51,75,220.379310
59,26,1,3,0.586207
15,70,56,227,419.758621
60,74,1,8,0.000000
0,19,55,625,2783.655172
22,38,55,310,528.896552


#### Number of item sold of category last n months

In [39]:
months = [max_train_month - i for i in range(3)]
months

[28, 27, 26]

In [40]:
df_ts.head()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,item_category_type,item_category_subtype,shop_city
0,5037,5,0,0.0,19,Игры,PS3,Вологда
1,5037,5,1,0.0,19,Игры,PS3,Вологда
2,5037,5,2,0.0,19,Игры,PS3,Вологда
3,5037,5,3,0.0,19,Игры,PS3,Вологда
4,5037,5,4,0.0,19,Игры,PS3,Вологда


In [41]:
print(len(categories_features))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby("item_category_id").agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "item_category_id_":"item_category_id",
        "item_cnt_month_sum":"item_category_sum_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"item_category_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"item_category_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"item_category_std_shifted_" + str(month_to_predict - month) + "_months"        
    },
    inplace=True)
    categories_features = pd.merge(categories_features, items_sold_in_month, on="item_category_id", how="left")
print(len(categories_features))
categories_features.head()

62
62


,item_category_id,total_shops_of_category,total_items_of_category,mean_sales_of_category_by_month,item_category_mean_shifted_1_months,item_category_sum_shifted_1_months,item_category_max_shifted_1_months,item_category_std_shifted_1_months,item_category_mean_shifted_2_months,item_category_sum_shifted_2_months,item_category_max_shifted_2_months,item_category_std_shifted_2_months,item_category_mean_shifted_3_months,item_category_sum_shifted_3_months,item_category_max_shifted_3_months,item_category_std_shifted_3_months
0,19,55,625,2783.655172,0.481463,2831.0,16.0,1.165208,0.379252,2230.0,15.0,0.872387,0.475510,2796.0,17.0,1.076783
1,55,53,2327,5192.379310,0.189590,5351.0,19.0,0.590362,0.207837,5866.0,20.0,0.692047,0.238733,6738.0,20.0,0.745403
2,23,55,496,2502.758621,0.331885,2063.0,17.0,0.902515,0.278314,1730.0,12.0,0.788954,0.383044,2381.0,18.0,0.995644
3,20,52,157,1921.931034,0.647376,3725.0,20.0,2.279662,0.549531,3162.0,20.0,2.136177,0.682829,3929.0,20.0,2.247666
4,30,55,751,5739.827586,0.915616,5230.0,20.0,2.171887,1.085084,6198.0,20.0,2.926780,0.986870,5637.0,20.0,2.026960


In [42]:
categories_features.to_csv("generated/" + features_folder_name + "features_categories.csv", index=False)

### Of category type (item_category_type)

In [43]:
category_types_features = df_aspects[["item_category_type"]].drop_duplicates(subset=["item_category_type"])
print(len(category_types_features))
category_types_features.head()

13


,item_category_type
29,Игры
4229,Музыка
37769,Подарки
43529,Карты
43649,Аксессуары


#### Historical mean of type

In [44]:
sum_of_sales_of_category_type = df_ts.groupby(["item_category_type", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_sales_of_category_type = sum_of_sales_of_category_type.groupby("item_category_type").agg({"item_cnt_month":"mean"}).reset_index().rename(columns={"item_cnt_month":"mean_sales_of_type_by_month"})
category_types_features = pd.merge(category_types_features, mean_sales_of_category_type, on="item_category_type", how="left")
print(len(category_types_features))
category_types_features.head()

13


,item_category_type,mean_sales_of_type_by_month
0,Игры,15800.551724
1,Музыка,5804.000000
2,Подарки,5177.413793
3,Карты,784.068966
4,Аксессуары,1321.000000


#### Item category type shifted n months

In [45]:
months = [max_train_month - i for i in range(3)]
months

[28, 27, 26]

In [46]:
print(len(category_types_features))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby("item_category_type").agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "item_category_type_":"item_category_type",
        "item_cnt_month_sum":"item_category_type_sum_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"item_category_type_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"item_category_type_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"item_category_type_std_shifted_" + str(month_to_predict - month) + "_months"        
    },
    inplace=True)
    print(len(category_types_features))
    category_types_features = pd.merge(category_types_features, items_sold_in_month, on="item_category_type", how="left")
category_types_features.head()

13
13
13
13


,item_category_type,mean_sales_of_type_by_month,item_category_type_mean_shifted_1_months,item_category_type_sum_shifted_1_months,item_category_type_max_shifted_1_months,item_category_type_std_shifted_1_months,item_category_type_mean_shifted_2_months,item_category_type_sum_shifted_2_months,item_category_type_max_shifted_2_months,item_category_type_std_shifted_2_months,item_category_type_mean_shifted_3_months,item_category_type_sum_shifted_3_months,item_category_type_max_shifted_3_months,item_category_type_std_shifted_3_months
0,Игры,15800.551724,0.403503,18269.0,20.0,1.496924,0.374945,16976.0,20.0,1.538700,0.416313,18849.0,20.0,1.367024
1,Музыка,5804.000000,0.134014,5865.0,19.0,0.497486,0.149072,6524.0,20.0,0.583057,0.170505,7462.0,20.0,0.627422
2,Подарки,5177.413793,0.181753,7626.0,20.0,0.931221,0.163020,6840.0,20.0,0.884639,0.201106,8438.0,20.0,0.976340
3,Карты,784.068966,1.523810,832.0,20.0,3.429551,1.659341,906.0,20.0,3.584533,2.457875,1342.0,20.0,4.885014
4,Аксессуары,1321.000000,0.356825,1124.0,20.0,1.083220,0.398095,1254.0,18.0,1.070519,0.412381,1299.0,9.0,0.950225


In [47]:
category_types_features.to_csv("generated/" + features_folder_name + "features_category_types.csv", index=False)

### Of category subtype (item_category_subtype)

In [48]:
category_subtypes_features = df_aspects[["item_category_subtype"]].drop_duplicates(subset=["item_category_subtype"])
print(len(category_subtypes_features))
category_subtypes_features.head()

49


,item_category_subtype
29,PS3
4229,CD локального производства
24389,XBOX 360
28829,PS4
32939,Стандартные издания


#### Historical mean of subtype

In [49]:
sum_of_sales_of_category_subtype = df_ts.groupby(["item_category_subtype", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_sales_of_category_subtype = sum_of_sales_of_category_subtype.groupby("item_category_subtype").agg({"item_cnt_month":"mean"}).reset_index().rename(columns={"item_cnt_month":"mean_sales_of_subtype_by_month"})
print(len(category_subtypes_features))
category_subtypes_features = pd.merge(category_subtypes_features, mean_sales_of_category_subtype, on="item_category_subtype", how="left")
print(len(category_subtypes_features))
category_subtypes_features.head()

49
49


,item_category_subtype,mean_sales_of_subtype_by_month
0,PS3,3007.724138
1,CD локального производства,5192.379310
2,XBOX 360,2955.965517
3,PS4,2627.413793
4,Стандартные издания,5739.827586


#### Item category subtype shifted n months

In [50]:
print(len(category_subtypes_features))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby("item_category_subtype").agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "item_category_subtype_":"item_category_subtype",
        "item_cnt_month_sum":"item_category_subtype_sum_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"item_category_subtype_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"item_category_subtype_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"item_category_subtype_std_shifted_" + str(month_to_predict - month) + "_months"        
    },
    inplace=True)
    print(len(category_subtypes_features))
    category_subtypes_features = pd.merge(category_subtypes_features, items_sold_in_month, on="item_category_subtype", how="left")
category_subtypes_features.head()

49
49
49
49


,item_category_subtype,mean_sales_of_subtype_by_month,item_category_subtype_mean_shifted_1_months,item_category_subtype_sum_shifted_1_months,item_category_subtype_max_shifted_1_months,item_category_subtype_std_shifted_1_months,item_category_subtype_mean_shifted_2_months,item_category_subtype_sum_shifted_2_months,item_category_subtype_max_shifted_2_months,item_category_subtype_std_shifted_2_months,item_category_subtype_mean_shifted_3_months,item_category_subtype_sum_shifted_3_months,item_category_subtype_max_shifted_3_months,item_category_subtype_std_shifted_3_months
0,PS3,3007.724138,0.466257,2957.0,16.0,1.136641,0.374488,2375.0,15.0,0.869283,0.468149,2969.0,17.0,1.062401
1,CD локального производства,5192.379310,0.189590,5351.0,19.0,0.590362,0.207837,5866.0,20.0,0.692047,0.238733,6738.0,20.0,0.745403
2,XBOX 360,2955.965517,0.337715,2312.0,17.0,0.909426,0.288197,1973.0,12.0,0.809222,0.393076,2691.0,18.0,1.008310
3,PS4,2627.413793,0.594156,4392.0,20.0,2.114964,0.523133,3867.0,20.0,1.975634,0.629870,4656.0,20.0,2.060292
4,Стандартные издания,5739.827586,0.915616,5230.0,20.0,2.171887,1.085084,6198.0,20.0,2.926780,0.986870,5637.0,20.0,2.026960


In [51]:
category_subtypes_features.to_csv("generated/" + features_folder_name + "features_category_subtypes.csv", index=False)

### Of shops (shop_id)

In [52]:
shops_features = df_aspects[["shop_id"]].drop_duplicates(subset=["shop_id"])

#### Total categories by shop

In [53]:
t_cat_by_shop = df.groupby("shop_id").agg({"item_category_id":"nunique"}).reset_index().rename(columns={"item_category_id":"tot_cat_by_shops"})
t_cat_by_shop.head()

print(len(shops_features))
shops_features = pd.merge(shops_features, t_cat_by_shop, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(10)

42
42


,shop_id,tot_cat_by_shops
18,14,59.0
4,2,58.0
22,49,53.0
2,6,63.0
3,3,57.0
40,39,54.0
11,25,65.0
7,12,63.0
38,41,58.0
9,31,62.0


#### Mean items sold by shop in the last month

In [54]:
items_sold_last_month = df_ts[df_ts["date_block_num"] == max_train_month].groupby("shop_id").agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
items_sold_last_month.columns = ['_'.join(col).strip() for col in items_sold_last_month.columns.values]

items_sold_last_month.rename(columns={
    "shop_id_":"shop_id",
    "item_cnt_month_mean":"mean_sales_by_shop_last_month",
    "item_cnt_month_sum":"sum_sales_by_shop_last_month",
    "item_cnt_month_max":"max_sales_by_shop_last_month",
    "item_cnt_month_std":"std_sales_by_shop_last_month"
    
}, inplace=True)

print(len(shops_features))
shops_features = pd.merge(shops_features, items_sold_last_month, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(5)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month
22,49,53.0,0.116275,593.0,20.0,0.679751
5,7,60.0,0.185882,948.0,20.0,0.887668
9,31,62.0,0.809412,4128.0,20.0,2.083600
40,39,54.0,0.113333,578.0,20.0,0.696909
36,34,49.0,0.065882,336.0,20.0,0.537268


#### Number of items sold by shop shifted n different months

In [55]:
len(df_ts[(df_ts.shop_id == 5) & (df_ts.date_block_num == 31)])

0

In [56]:
months = [month_to_predict - 1 - i for i in range(2)]
months

[28, 27]

In [57]:
print(len(shops_features))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby("shop_id").agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "shop_id_":"shop_id",
        "item_cnt_month_sum":"items_sold_by_shop_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"shop_item_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"shop_item_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"shop_item_std_shifted_" + str(month_to_predict - month) + "_months"        
    },
    inplace=True)
    shops_features = pd.merge(shops_features, items_sold_in_month, on="shop_id", how="left")
print(len(shops_features))

42
42


In [58]:
shops_features.head()

,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months
0,5,58.0,0.163333,833.0,20.0,0.803862,0.163333,833.0,20.0,0.803862,0.139216,710.0,20.0,0.772791
1,4,60.0,0.133529,681.0,20.0,0.746577,0.133529,681.0,20.0,0.746577,0.126275,644.0,20.0,0.767894
2,6,63.0,0.228235,1164.0,20.0,0.976517,0.228235,1164.0,20.0,0.976517,0.254314,1297.0,20.0,1.159319
3,3,57.0,0.118627,605.0,20.0,0.677899,0.118627,605.0,20.0,0.677899,0.116667,595.0,20.0,0.756949
4,2,58.0,0.118627,605.0,20.0,0.747255,0.118627,605.0,20.0,0.747255,0.111569,569.0,20.0,0.712910


#### Rolling features

In [59]:
items_sells_historically = df_ts.groupby(["shop_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
grouped = items_sells_historically.sort_values(by="date_block_num").groupby("shop_id").agg({"item_cnt_month":[rolling_mean, rolling_std, rolling_max, rolling_min, diffmean]}).reset_index()
grouped.columns = ['_'.join(col).strip() + "_shop" for col in grouped.columns.values]
grouped.rename(columns={"shop_id__shop":"shop_id"}, inplace=True)
grouped.head()

,shop_id,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop
0,2,476.753086,227.478497,949.000000,180.000000,12.750000
1,3,452.629630,175.134364,813.000000,207.666667,15.111111
2,4,630.888889,206.409642,1070.666667,243.333333,8.730769
3,5,628.987654,251.586173,1091.666667,183.000000,29.750000
4,6,1298.012346,367.813678,2034.666667,726.666667,12.857143


In [60]:
shops_features = pd.merge(shops_features, grouped, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(6)

42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop
4,2,58.0,0.118627,605.0,20.0,0.747255,0.118627,605.0,20.0,0.747255,0.111569,569.0,20.0,0.712910,476.753086,227.478497,949.000000,180.000000,12.750000
41,45,59.0,0.118627,605.0,20.0,0.666815,0.118627,605.0,20.0,0.666815,0.116275,593.0,20.0,0.735992,597.283951,131.869681,877.666667,374.666667,6.703704
30,55,11.0,0.174118,888.0,20.0,1.352645,0.174118,888.0,20.0,1.352645,0.164118,837.0,20.0,1.229311,510.222222,310.259748,1041.333333,0.000000,35.520000
0,5,58.0,0.163333,833.0,20.0,0.803862,0.163333,833.0,20.0,0.803862,0.139216,710.0,20.0,0.772791,628.987654,251.586173,1091.666667,183.000000,29.750000
9,31,62.0,0.809412,4128.0,20.0,2.083600,0.809412,4128.0,20.0,2.083600,0.866078,4417.0,20.0,2.249732,3502.160494,1206.882101,6065.000000,1999.666667,75.500000
25,47,60.0,0.285882,1458.0,20.0,1.155933,0.285882,1458.0,20.0,1.155933,0.262941,1341.0,20.0,1.116687,996.654321,485.261270,1917.666667,414.666667,37.178571


#### Total items sell by shop

In [61]:
tot_items_by_shop = df.groupby("shop_id").agg({"item_id":"nunique"}).reset_index().rename(columns={"item_id":"tot_items_by_shop"})
tot_items_by_shop.head()

print(len(shops_features))
shops_features = pd.merge(shops_features, tot_items_by_shop, on="shop_id", how="left")
shops_features["tot_items_by_shop"].fillna(0)
print(len(shops_features))
shops_features.sample(10)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop,tot_items_by_shop
11,25,65.0,0.692745,3533.0,20.0,1.906293,0.692745,3533.0,20.0,1.906293,0.711176,3627.0,20.0,2.003995,2683.716049,1011.938284,4878.666667,1470.666667,72.035714,14284.0
38,41,58.0,0.142353,726.0,20.0,0.781811,0.142353,726.0,20.0,0.781811,0.103529,528.0,20.0,0.725429,675.666667,173.748812,997.666667,414.666667,11.035714,6765.0
36,34,49.0,0.065882,336.0,20.0,0.537268,0.065882,336.0,20.0,0.537268,0.061765,315.0,20.0,0.522028,106.851852,153.853447,392.666667,0.000000,30.545455,1408.0
13,24,56.0,0.176471,900.0,20.0,0.912227,0.176471,900.0,20.0,0.912227,0.190980,974.0,20.0,1.008299,928.913580,301.602367,1559.000000,381.666667,19.538462,7495.0
33,37,61.0,0.138627,707.0,20.0,0.803380,0.138627,707.0,20.0,0.803380,0.144510,737.0,20.0,0.823333,620.358025,178.520293,1054.333333,385.333333,7.000000,7886.0
39,44,62.0,0.124706,636.0,20.0,0.701145,0.124706,636.0,20.0,0.701145,0.132745,677.0,20.0,0.730520,601.716049,136.709740,882.666667,378.333333,9.481481,8291.0
18,14,59.0,0.125882,642.0,20.0,0.741717,0.125882,642.0,20.0,0.741717,0.127647,651.0,20.0,0.845358,626.530864,187.292820,1035.333333,376.666667,7.285714,6808.0
25,47,60.0,0.285882,1458.0,20.0,1.155933,0.285882,1458.0,20.0,1.155933,0.262941,1341.0,20.0,1.116687,996.654321,485.261270,1917.666667,414.666667,37.178571,7244.0
2,6,63.0,0.228235,1164.0,20.0,0.976517,0.228235,1164.0,20.0,0.976517,0.254314,1297.0,20.0,1.159319,1298.012346,367.813678,2034.666667,726.666667,12.857143,11070.0
15,15,58.0,0.193137,985.0,20.0,0.951126,0.193137,985.0,20.0,0.951126,0.190784,973.0,20.0,1.065364,982.580247,289.354980,1582.666667,528.666667,13.464286,7680.0


#### Tot items by shop in last month

In [62]:
tot_items_by_shop_last_month = df[df.date_block_num == max_train_month].groupby("shop_id").agg({"item_id":"nunique"}).reset_index().rename(columns={"item_id":"tot_items_by_shop_last_month"})
tot_items_by_shop_last_month.head()

print(len(shops_features))
shops_features = pd.merge(shops_features, tot_items_by_shop_last_month, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(10)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop,tot_items_by_shop,tot_items_by_shop_last_month
37,46,60.0,0.228824,1167.0,20.0,1.006934,0.228824,1167.0,20.0,1.006934,0.206275,1052.0,20.0,0.972156,1068.753086,321.601755,1763.333333,585.333333,16.714286,8480.0,750.0
8,28,60.0,0.551961,2815.0,20.0,1.714767,0.551961,2815.0,20.0,1.714767,0.507255,2587.0,20.0,1.666010,2324.703704,811.586909,4045.666667,1202.333333,55.071429,11726.0,1457.0
26,48,51.0,0.176667,901.0,20.0,0.877227,0.176667,901.0,20.0,0.877227,0.165294,843.0,20.0,0.785940,441.160494,490.642413,1263.000000,0.000000,64.357143,3440.0,586.0
23,53,58.0,0.177843,907.0,20.0,0.810029,0.177843,907.0,20.0,0.810029,0.179804,917.0,20.0,0.893870,919.765432,271.023887,1440.000000,466.333333,15.535714,6815.0,669.0
21,50,57.0,0.164510,839.0,20.0,0.829323,0.164510,839.0,20.0,0.829323,0.130392,665.0,20.0,0.766310,1000.049383,211.981246,1422.333333,751.666667,2.821429,8591.0,624.0
13,24,56.0,0.176471,900.0,20.0,0.912227,0.176471,900.0,20.0,0.912227,0.190980,974.0,20.0,1.008299,928.913580,301.602367,1559.000000,381.666667,19.538462,7495.0,564.0
34,35,61.0,0.225882,1152.0,20.0,0.948754,0.225882,1152.0,20.0,0.948754,0.223922,1142.0,20.0,1.115446,964.740741,364.702482,1747.000000,458.666667,24.814815,8474.0,761.0
2,6,63.0,0.228235,1164.0,20.0,0.976517,0.228235,1164.0,20.0,0.976517,0.254314,1297.0,20.0,1.159319,1298.012346,367.813678,2034.666667,726.666667,12.857143,11070.0,860.0
24,52,59.0,0.140392,716.0,20.0,0.744659,0.140392,716.0,20.0,0.744659,0.157451,803.0,20.0,0.842414,710.370370,214.678817,1143.666667,382.000000,13.222222,7834.0,519.0
36,34,49.0,0.065882,336.0,20.0,0.537268,0.065882,336.0,20.0,0.537268,0.061765,315.0,20.0,0.522028,106.851852,153.853447,392.666667,0.000000,30.545455,1408.0,249.0


#### Historically mean sales by shop

In [63]:
df_ts.head()

,item_id,shop_id,date_block_num,item_cnt_month,item_category_id,item_category_type,item_category_subtype,shop_city
0,5037,5,0,0.0,19,Игры,PS3,Вологда
1,5037,5,1,0.0,19,Игры,PS3,Вологда
2,5037,5,2,0.0,19,Игры,PS3,Вологда
3,5037,5,3,0.0,19,Игры,PS3,Вологда
4,5037,5,4,0.0,19,Игры,PS3,Вологда


In [64]:
total_sum_sales_by_shop_and_month = df_ts.groupby(["shop_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_sales_by_shop = total_sum_sales_by_shop_and_month.groupby("shop_id").agg({"item_cnt_month":["mean", "std", "max", "min"]}).reset_index()
mean_sales_by_shop.columns = ['_'.join(col).strip() for col in mean_sales_by_shop.columns.values]

mean_sales_by_shop.rename(columns={
    "item_cnt_month_mean":"mean_sales_by_shop_and_month",
    "item_cnt_month_std":"std_sales_by_shop_and_month",
    "item_cnt_month_max":"max_sales_by_shop_and_month",
    "item_cnt_month_min":"min_sales_by_shop_and_month",
    "shop_id_":"shop_id"
}, inplace=True)

print(len(shops_features))
shops_features = pd.merge(shops_features, mean_sales_by_shop, on="shop_id", how="left")
print(len(shops_features))

shops_features.sample(8)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop,tot_items_by_shop,tot_items_by_shop_last_month,mean_sales_by_shop_and_month,std_sales_by_shop_and_month,max_sales_by_shop_and_month,min_sales_by_shop_and_month
24,52,59.0,0.140392,716.0,20.0,0.744659,0.140392,716.0,20.0,0.744659,0.157451,803.0,20.0,0.842414,710.370370,214.678817,1143.666667,382.000000,13.222222,7834.0,519.0,699.965517,253.618876,1600.0,359.0
3,3,57.0,0.118627,605.0,20.0,0.677899,0.118627,605.0,20.0,0.677899,0.116667,595.0,20.0,0.756949,452.629630,175.134364,813.000000,207.666667,15.111111,4812.0,457.0,449.000000,189.632916,933.0,180.0
23,53,58.0,0.177843,907.0,20.0,0.810029,0.177843,907.0,20.0,0.810029,0.179804,917.0,20.0,0.893870,919.765432,271.023887,1440.000000,466.333333,15.535714,6815.0,669.0,904.137931,305.424497,1792.0,387.0
0,5,58.0,0.163333,833.0,20.0,0.803862,0.163333,833.0,20.0,0.803862,0.139216,710.0,20.0,0.772791,628.987654,251.586173,1091.666667,183.000000,29.750000,6472.0,572.0,615.068966,289.988045,1418.0,0.0
17,18,62.0,0.196863,1004.0,20.0,0.954788,0.196863,1004.0,20.0,0.954788,0.189608,967.0,20.0,0.980857,840.074074,176.671996,1178.000000,592.666667,11.888889,8404.0,664.0,840.482759,223.952741,1480.0,468.0
29,59,57.0,0.145686,743.0,20.0,0.778808,0.145686,743.0,20.0,0.778808,0.148627,758.0,20.0,0.904039,697.827160,208.950253,1137.000000,379.666667,10.703704,6821.0,506.0,691.034483,234.883023,1402.0,333.0
32,36,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0
36,34,49.0,0.065882,336.0,20.0,0.537268,0.065882,336.0,20.0,0.537268,0.061765,315.0,20.0,0.522028,106.851852,153.853447,392.666667,0.000000,30.545455,1408.0,249.0,110.827586,161.184692,481.0,0.0


#### Number of months in which shop has sales

In [65]:
total_months_with_sales = df.groupby("shop_id").agg({"date_block_num":"nunique"}).reset_index().rename(columns={"date_block_num":"months_with_sales_by_shop"})
print(len(shops_features))
shops_features = pd.merge(shops_features, total_months_with_sales, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(5)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop,tot_items_by_shop,tot_items_by_shop_last_month,mean_sales_by_shop_and_month,std_sales_by_shop_and_month,max_sales_by_shop_and_month,min_sales_by_shop_and_month,months_with_sales_by_shop
31,56,62.0,0.192745,983.0,20.0,0.809525,0.192745,983.0,20.0,0.809525,0.174118,888.0,20.0,0.791376,973.827160,284.860490,1592.000000,556.000000,14.428571,10348.0,835.0,958.655172,325.224898,1800.0,485.0,29.0
21,50,57.0,0.164510,839.0,20.0,0.829323,0.164510,839.0,20.0,0.829323,0.130392,665.0,20.0,0.766310,1000.049383,211.981246,1422.333333,751.666667,2.821429,8591.0,624.0,983.206897,264.816969,1815.0,665.0,29.0
5,7,60.0,0.185882,948.0,20.0,0.887668,0.185882,948.0,20.0,0.887668,0.190588,972.0,20.0,0.936789,966.580247,347.971528,1722.333333,442.333333,14.071429,7690.0,632.0,952.379310,388.000774,2261.0,386.0,29.0
7,12,63.0,0.157451,803.0,20.0,1.124228,0.157451,803.0,20.0,1.124228,0.215686,1100.0,20.0,1.281879,639.024691,305.968197,1292.333333,279.000000,17.357143,6594.0,487.0,637.241379,340.237276,1783.0,234.0,29.0
4,2,58.0,0.118627,605.0,20.0,0.747255,0.118627,605.0,20.0,0.747255,0.111569,569.0,20.0,0.712910,476.753086,227.478497,949.000000,180.000000,12.750000,4349.0,409.0,471.586207,251.486512,1338.0,136.0,29.0


In [66]:
df.head()

,item_id,item_category_id,item_category_type,item_category_subtype,date_block_num,shop_id,item_price,item_cnt_day,shop_city
0,0,40,Кино,DVD,20,54,58.0,1.0,Химки
1,2,40,Кино,DVD,19,54,58.0,1.0,Химки
2,2,40,Кино,DVD,22,54,58.0,1.0,Химки
3,3,40,Кино,DVD,18,54,100.0,1.0,Химки
4,3,40,Кино,DVD,19,54,58.0,1.0,Химки


#### Mean prices of items sale by shop

In [67]:
prices_of_items_sold_by_shop = df.groupby("shop_id").agg({"item_price":["mean", "std", "min", "max"]}).reset_index()
prices_of_items_sold_by_shop.columns = ['_'.join(col).strip() for col in prices_of_items_sold_by_shop.columns.values]
prices_of_items_sold_by_shop.rename(columns={
    "item_price_mean":"item_price_mean_by_shop",
    "item_price_std":"item_price_std_by_shop",
    "item_price_max":"item_price_max_by_shop",
    "item_price_min":"item_price_min_by_shop",
    "shop_id_":"shop_id"
}, inplace=True)

print(len(shops_features))
shops_features = pd.merge(shops_features, prices_of_items_sold_by_shop, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(5)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop,tot_items_by_shop,tot_items_by_shop_last_month,mean_sales_by_shop_and_month,std_sales_by_shop_and_month,max_sales_by_shop_and_month,min_sales_by_shop_and_month,months_with_sales_by_shop,item_price_mean_by_shop,item_price_std_by_shop,item_price_min_by_shop,item_price_max_by_shop
9,31,62.0,0.809412,4128.0,20.0,2.083600,0.809412,4128.0,20.0,2.083600,0.866078,4417.0,20.0,2.249732,3502.160494,1206.882101,6065.000000,1999.666667,75.500000,13331.0,2285.0,3478.551724,1340.330412,8065.0,1983.0,29.0,708.638169,1354.567556,0.1,31490.0
21,50,57.0,0.164510,839.0,20.0,0.829323,0.164510,839.0,20.0,0.829323,0.130392,665.0,20.0,0.766310,1000.049383,211.981246,1422.333333,751.666667,2.821429,8591.0,624.0,983.206897,264.816969,1815.0,665.0,29.0,952.271865,1826.407047,0.1,28490.0
4,2,58.0,0.118627,605.0,20.0,0.747255,0.118627,605.0,20.0,0.747255,0.111569,569.0,20.0,0.712910,476.753086,227.478497,949.000000,180.000000,12.750000,4349.0,409.0,471.586207,251.486512,1338.0,136.0,29.0,1351.385492,2221.294382,0.1,29990.0
18,14,59.0,0.125882,642.0,20.0,0.741717,0.125882,642.0,20.0,0.741717,0.127647,651.0,20.0,0.845358,626.530864,187.292820,1035.333333,376.666667,7.285714,6808.0,486.0,620.896552,208.778821,1272.0,268.0,29.0,955.165322,1774.469256,0.1,36990.0
32,36,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,NaN,NaN,NaN,NaN,NaN


#### Shop renevue trend

In [68]:
df_shops = df_aux.copy()
df_shops.head()

,item_name,item_id,item_category_id,item_category_name,item_category_type,item_category_subtype,date,date_block_num,shop_id,item_price,item_cnt_day,shop_name,shop_city
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино,DVD,01.09.2014,20,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,24.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,12.11.2014,22,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,05.07.2014,18,54,100.0,1.0,"Химки ТЦ ""Мега""",Химки
4,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,26.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки


In [69]:
df_shops["date_block_num"].max()

33

In [70]:
df_shops["revenue"] = df_shops["item_price"] * df_shops["item_cnt_day"]
df_shops.head()

,item_name,item_id,item_category_id,item_category_name,item_category_type,item_category_subtype,date,date_block_num,shop_id,item_price,item_cnt_day,shop_name,shop_city,revenue
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино,DVD,01.09.2014,20,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки,58.0
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,24.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки,58.0
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,12.11.2014,22,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки,58.0
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,05.07.2014,18,54,100.0,1.0,"Химки ТЦ ""Мега""",Химки,100.0
4,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,26.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки,58.0


In [71]:
df_shops["revenue"] = df_shops["revenue"].astype(np.float32)
group = df_shops.groupby(["shop_id", "date_block_num"]).agg({"revenue":"sum"}).reset_index().rename(columns={"revenue":"month_revenue"})

In [72]:
group["month_revenue"] = group["month_revenue"].astype(np.float32)
s_group = group.groupby("shop_id").agg({"month_revenue":"mean"}).reset_index().rename(columns={"month_revenue":"month_revenue_mean"})
s_group.head()

,shop_id,month_revenue_mean
0,2,1.295578e+06
1,3,8.864957e+05
2,4,1.192343e+06
3,5,1.156377e+06
4,6,2.884337e+06


In [73]:
last_month_revenue = df_shops[df_shops.date_block_num == max_train_month].groupby("shop_id").agg({"revenue":"sum"}).reset_index().rename(columns={"revenue":"last_month_revenue"})
last_month_revenue.head()

,shop_id,last_month_revenue
0,2,1470399.0
1,3,1146003.0
2,4,981594.0
3,5,1036892.0
4,6,2683205.0


In [74]:
merged = pd.merge(s_group, last_month_revenue, on="shop_id", how="inner")
merged.head()

,shop_id,month_revenue_mean,last_month_revenue
0,2,1.295578e+06,1470399.0
1,3,8.864957e+05,1146003.0
2,4,1.192343e+06,981594.0
3,5,1.156377e+06,1036892.0
4,6,2.884337e+06,2683205.0


In [75]:
merged["month_revenue_mean"] = merged["month_revenue_mean"].astype(np.float32) 

In [76]:
merged.head()

,shop_id,month_revenue_mean,last_month_revenue
0,2,1.295578e+06,1470399.0
1,3,8.864957e+05,1146003.0
2,4,1.192343e+06,981594.0
3,5,1.156377e+06,1036892.0
4,6,2.884337e+06,2683205.0


In [77]:
merged["delta_revenue"] = (merged["last_month_revenue"] - merged["month_revenue_mean"]) / merged["month_revenue_mean"]
merged.head()

,shop_id,month_revenue_mean,last_month_revenue,delta_revenue
0,2,1.295578e+06,1470399.0,0.134937
1,3,8.864957e+05,1146003.0,0.292734
2,4,1.192343e+06,981594.0,-0.176752
3,5,1.156377e+06,1036892.0,-0.103327
4,6,2.884337e+06,2683205.0,-0.069732


In [78]:
merged.drop(columns=["month_revenue_mean", "last_month_revenue"], inplace=True)
print(len(shops_features))
shops_features = pd.merge(shops_features, merged, on="shop_id", how="left")
print(len(shops_features))
shops_features.sample(5)

42
42


,shop_id,tot_cat_by_shops,mean_sales_by_shop_last_month,sum_sales_by_shop_last_month,max_sales_by_shop_last_month,std_sales_by_shop_last_month,shop_item_mean_shifted_1_months,items_sold_by_shop_shifted_1_months,shop_item_max_shifted_1_months,shop_item_std_shifted_1_months,shop_item_mean_shifted_2_months,items_sold_by_shop_shifted_2_months,shop_item_max_shifted_2_months,shop_item_std_shifted_2_months,item_cnt_month_rolling_mean_shop,item_cnt_month_rolling_std_shop,item_cnt_month_rolling_max_shop,item_cnt_month_rolling_min_shop,item_cnt_month_diffmean_shop,tot_items_by_shop,tot_items_by_shop_last_month,mean_sales_by_shop_and_month,std_sales_by_shop_and_month,max_sales_by_shop_and_month,min_sales_by_shop_and_month,months_with_sales_by_shop,item_price_mean_by_shop,item_price_std_by_shop,item_price_min_by_shop,item_price_max_by_shop,delta_revenue
41,45,59.0,0.118627,605.0,20.0,0.666815,0.118627,605.0,20.0,0.666815,0.116275,593.0,20.0,0.735992,597.283951,131.869681,877.666667,374.666667,6.703704,6729.0,484.0,591.068966,164.472736,1114.0,343.0,29.0,926.145696,1769.648144,0.1,30490.0,-0.215732
20,42,62.0,0.499804,2549.0,20.0,1.670728,0.499804,2549.0,20.0,1.670728,0.519020,2647.0,20.0,1.792160,1685.950617,706.639602,3069.666667,836.666667,60.178571,11076.0,1433.0,1688.620690,786.569333,4116.0,793.0,29.0,908.737434,1736.217197,0.1,33490.0,0.473828
31,56,62.0,0.192745,983.0,20.0,0.809525,0.192745,983.0,20.0,0.809525,0.174118,888.0,20.0,0.791376,973.827160,284.860490,1592.000000,556.000000,14.428571,10348.0,835.0,958.655172,325.224898,1800.0,485.0,29.0,725.516740,1507.994752,0.1,32990.0,-0.232861
17,18,62.0,0.196863,1004.0,20.0,0.954788,0.196863,1004.0,20.0,0.954788,0.189608,967.0,20.0,0.980857,840.074074,176.671996,1178.000000,592.666667,11.888889,8404.0,664.0,840.482759,223.952741,1480.0,468.0,29.0,1165.925616,2097.486253,0.1,28990.0,-0.104494
30,55,11.0,0.174118,888.0,20.0,1.352645,0.174118,888.0,20.0,1.352645,0.164118,837.0,20.0,1.229311,510.222222,310.259748,1041.333333,0.000000,35.520000,1794.0,422.0,505.068966,327.736385,1093.0,0.0,25.0,572.764719,778.336558,4.0,27900.0,0.066611


In [79]:
shops_features.to_csv("generated/" + features_folder_name +"features_shops.csv", index=False)

### Of cities

In [80]:
cities_features = df_aspects[["shop_city"]].drop_duplicates(subset=["shop_city"])
print(len(cities_features))
cities_features.head()

28


,shop_city
29,Вологда
153029,Волжский
306029,Воронеж
459029,Балашиха
612029,Адыгея


In [81]:
df_ts_cities = df_ts.copy()

#### Number of shops placed in a city

In [82]:
total_shops_by_city = df.groupby("shop_city").agg({"shop_id":"nunique"}).reset_index().rename(columns={"shop_id":"tot_shops_by_city"})
print(len(cities_features))
cities_features = pd.merge(cities_features, total_shops_by_city, on="shop_city", how="left")
print(len(cities_features))
cities_features.head()

28
28


,shop_city,tot_shops_by_city
0,Вологда,1
1,Волжский,1
2,Воронеж,3
3,Балашиха,1
4,Адыгея,1


#### Number of items that are sell in a city

In [83]:
total_items_by_city = df.groupby("shop_city").agg({"item_id":"nunique"}).reset_index().rename(columns={"item_id":"tot_items_by_city"})
cities_features = pd.merge(cities_features, total_items_by_city, on="shop_city", how="left")
print(len(cities_features))
cities_features.sample(5)

28


,shop_city,tot_shops_by_city,tot_items_by_city
11,Казань,2,8517
1,Волжский,1,6787
26,РостовНаДону,3,7656
17,Томск,1,3440
15,Уфа,2,9272


#### Mean sales by city

In [84]:
items_sold_in_city_by_month = df_ts_cities.groupby(["shop_city", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_sales_in_cities = items_sold_in_city_by_month.groupby("shop_city").agg({"item_cnt_month":["mean", "max", "min", "std", "sum"]}).reset_index()

mean_sales_in_cities.columns = ['_'.join(col).strip() for col in mean_sales_in_cities.columns.values]

mean_sales_in_cities.rename(columns={
    "item_cnt_month_mean":"mean_sales_by_city",
    "item_cnt_month_std":"std_sales_by_city",
    "item_cnt_month_max":"max_sales_by_city",
    "item_cnt_month_min":"min_sales_by_city",
    "item_cnt_month_sum":"sum_sales_by_city",
    "shop_city_":"shop_city"
}, inplace=True)

cities_features = pd.merge(cities_features, mean_sales_in_cities, on="shop_city", how="left")
print(len(cities_features))

cities_features.sample(5)

28


,shop_city,tot_shops_by_city,tot_items_by_city,mean_sales_by_city,max_sales_by_city,min_sales_by_city,std_sales_by_city,sum_sales_by_city
10,Красноярск,2,9314,840.482759,1480.0,468.0,223.952741,24374.0
4,Адыгея,1,4349,471.586207,1338.0,136.0,251.486512,13676.0
22,Новосибирск,1,7886,620.586207,1356.0,320.0,206.122973,17997.0
23,Н.Новгород,2,8594,1065.793103,2845.0,446.0,550.224265,30908.0
15,Уфа,2,9272,1604.103448,3392.0,747.0,553.567220,46519.0


#### Mean sales in city last month

In [85]:
items_sold_in_city_last_month = df_ts_cities[df_ts_cities["date_block_num"] == max_train_month].groupby(["shop_city"]).agg({"item_cnt_month":["mean", "max", "sum"]}).reset_index()

In [86]:
items_sold_in_city_last_month.columns = ['_'.join(col).strip() for col in items_sold_in_city_last_month.columns.values]

In [87]:
items_sold_in_city_last_month.head()

,shop_city_,item_cnt_month_mean,item_cnt_month_max,item_cnt_month_sum
0,Адыгея,0.118627,20.0,605.0
1,Балашиха,0.118627,20.0,605.0
2,Волжский,0.133529,20.0,681.0
3,Вологда,0.163333,20.0,833.0
4,Воронеж,0.207059,20.0,2112.0


In [88]:
items_sold_in_city_last_month.rename(columns={
    "item_cnt_month_mean":"mean_sales_by_city_last_month",
    "item_cnt_month_std":"std_sales_by_city_last_month",
    "item_cnt_month_max":"max_sales_by_city_last_month",
    "item_cnt_month_min":"min_sales_by_city_last_month",
    "item_cnt_month_sum":"sum_sales_by_city_last_month",
    "shop_city_":"shop_city"
}, inplace=True)

cities_features = pd.merge(cities_features, items_sold_in_city_last_month, on="shop_city", how="left")
print(len(cities_features))

28


In [89]:
cities_features.sample(5)

,shop_city,tot_shops_by_city,tot_items_by_city,mean_sales_by_city,max_sales_by_city,min_sales_by_city,std_sales_by_city,sum_sales_by_city,mean_sales_by_city_last_month,max_sales_by_city_last_month,sum_sales_by_city_last_month
9,Коломна,1,8842,829.275862,1823.0,536.0,275.388284,24049.0,0.182941,20.0,933.0
24,Омск,1,6787,797.724138,1979.0,370.0,362.916647,23134.0,0.204510,20.0,1043.0
13,СПб,2,11583,1688.620690,4116.0,793.0,786.569333,48970.0,0.499804,20.0,2549.0
1,Волжский,1,6787,626.379310,1438.0,63.0,239.259783,18165.0,0.133529,20.0,681.0
11,Казань,2,8517,620.896552,1272.0,268.0,208.778821,18006.0,0.125882,20.0,642.0


In [90]:
cities_features.to_csv("generated/" + features_folder_name +"features_cities.csv", index=False)

### By item in specific shop (item_id & shop_id)

In [91]:
features_items_and_shops = df_aspects[["item_id", "shop_id"]].drop_duplicates(subset=["item_id", "shop_id"])
print(len(features_items_and_shops))
features_items_and_shops.head()

214200


,item_id,shop_id
29,5037,5
59,5233,5
89,4872,5
119,5002,5
149,5814,5


#### Items sold in shop last month

In [92]:
item_purchases_by_shop_last_month = df_ts[df_ts["date_block_num"] == max_train_month].groupby(["shop_id", "item_id"]).agg({"item_cnt_month":"sum"}).reset_index().rename(columns={"item_cnt_month":"purchases_shop_last_month"})

features_items_and_shops = pd.merge(features_items_and_shops, item_purchases_by_shop_last_month, on=["item_id", "shop_id"], how="left")

print(len(features_items_and_shops))
features_items_and_shops.sample(10)

214200


,item_id,shop_id,purchases_shop_last_month
24594,13285,2,0.0
173192,10225,37,0.0
58367,13946,25,0.0
137692,5386,48,0.0
144464,6395,58,0.0
20103,13134,3,0.0
72953,8502,21,0.0
3156,5191,5,0.0
50474,5063,31,0.0
169013,9550,37,0.0


#### Item sold in shop in last n months

In [93]:
months = [max_train_month - 1 - i for i in range(2)]
months

[27, 26]

In [94]:
for month in months:
    item_purchases_by_shop_in_month = df_ts[df_ts["date_block_num"] == month].groupby(["shop_id", "item_id"]).agg({"item_cnt_month":"sum"}).reset_index().rename(columns={"item_cnt_month":"purchases_item_in_shop_month_" + str(month_to_predict - month)})
    features_items_and_shops = pd.merge(features_items_and_shops, item_purchases_by_shop_in_month, on=["shop_id", "item_id"], how="left")
    print(len(features_items_and_shops))

214200
214200


In [95]:
features_items_and_shops.head()

,item_id,shop_id,purchases_shop_last_month,purchases_item_in_shop_month_2,purchases_item_in_shop_month_3
0,5037,5,1.0,0.0,0.0
1,5233,5,3.0,0.0,0.0
2,4872,5,6.0,1.0,7.0
3,5002,5,0.0,0.0,0.0
4,5814,5,1.0,0.0,1.0


#### Historical sales for shop in specific item (sum, mean, std, min, max)

In [96]:
sales_of_item_in_shop_by_month_h = df_ts.groupby(["item_id", "shop_id"]).agg({"item_cnt_month": ["mean", "sum", "std", "max", "min"]}).reset_index().rename(columns={"sales_in_month":"average_sales_for_item_and_shop_by_month"})

sales_of_item_in_shop_by_month_h.columns = ['_'.join(col).strip() for col in sales_of_item_in_shop_by_month_h.columns.values]

sales_of_item_in_shop_by_month_h.rename(columns={
   "shop_id_":"shop_id",
    "item_id_":"item_id",
    "item_cnt_month_sum":"hist_sales_sum_item_by_shop",
    "item_cnt_month_mean":"hist_sales_mean_item_by_shop",
    "item_cnt_month_std":"hist_sales_std_item_by_shop",
    "item_cnt_month_min":"hist_sales_min_item_by_shop",
    "item_cnt_month_max":"hist_sales_max_item_by_shop"
}, inplace=True)

print(len(features_items_and_shops))
print(len(sales_of_item_in_shop_by_month_h))
features_items_and_shops = pd.merge(features_items_and_shops, sales_of_item_in_shop_by_month_h, on=["shop_id", "item_id"],how="left")
print(len(features_items_and_shops))

features_items_and_shops.sample(10)

214200
214200
214200


,item_id,shop_id,purchases_shop_last_month,purchases_item_in_shop_month_2,purchases_item_in_shop_month_3,hist_sales_mean_item_by_shop,hist_sales_sum_item_by_shop,hist_sales_std_item_by_shop,hist_sales_max_item_by_shop,hist_sales_min_item_by_shop
113808,6421,49,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
142237,8112,57,0.0,0.0,0.0,0.034483,1.0,0.185695,1.0,0.0
139835,10093,57,3.0,2.0,0.0,0.172414,5.0,0.658405,3.0,0.0
56299,10914,25,0.0,3.0,0.0,0.482759,14.0,0.784706,3.0,0.0
41963,12134,28,2.0,1.0,2.0,3.862069,112.0,2.488892,10.0,1.0
47226,15922,31,1.0,0.0,0.0,0.724138,21.0,1.130630,4.0,0.0
134057,5638,48,0.0,0.0,0.0,0.034483,1.0,0.185695,1.0,0.0
182965,13030,38,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
93768,8733,14,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
61170,656,25,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


#### Last ten months sales for shop in specific item (sum, mean, std, min, max) 

In [97]:
sales_of_item_in_shop_by_month_h_10 = df_ts[df_ts.date_block_num >= max_train_month - 10].groupby(["item_id", "shop_id"]).agg({"item_cnt_month": ["mean", "sum", "std", "max", "min"]}).reset_index().rename(columns={"sales_in_month":"average_sales_for_item_and_shop_by_month"})

sales_of_item_in_shop_by_month_h_10.columns = ['_'.join(col).strip() for col in sales_of_item_in_shop_by_month_h_10.columns.values]

sales_of_item_in_shop_by_month_h_10.rename(columns={
   "shop_id_":"shop_id",
    "item_id_":"item_id",
    "item_cnt_month_sum":"ten_month_sales_sum_item_by_shop",
    "item_cnt_month_mean":"ten_month_mean_item_by_shop",
    "item_cnt_month_std":"ten_month_std_item_by_shop",
    "item_cnt_month_min":"ten_month_min_item_by_shop",
    "item_cnt_month_max":"ten_month_max_item_by_shop"
}, inplace=True)

print(len(features_items_and_shops))
print(len(sales_of_item_in_shop_by_month_h_10))
features_items_and_shops = pd.merge(features_items_and_shops, sales_of_item_in_shop_by_month_h_10, on=["shop_id", "item_id"],how="left")
print(len(features_items_and_shops))

features_items_and_shops.sample(10)

214200
214200
214200


,item_id,shop_id,purchases_shop_last_month,purchases_item_in_shop_month_2,purchases_item_in_shop_month_3,hist_sales_mean_item_by_shop,hist_sales_sum_item_by_shop,hist_sales_std_item_by_shop,hist_sales_max_item_by_shop,hist_sales_min_item_by_shop,ten_month_mean_item_by_shop,ten_month_sales_sum_item_by_shop,ten_month_std_item_by_shop,ten_month_max_item_by_shop,ten_month_min_item_by_shop
113474,20448,49,0.0,0.0,0.0,0.413793,12.0,0.982607,4.0,0.0,1.000000,11.0,1.414214,4.0,0.0
151420,4767,59,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
180849,10747,38,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
106460,3080,42,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
62906,1580,22,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
127763,12543,47,0.0,0.0,0.0,0.172414,5.0,0.384426,1.0,0.0,0.090909,1.0,0.301511,1.0,0.0
24895,3797,2,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
17705,11768,3,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
168959,1256,37,0.0,0.0,0.0,0.103448,3.0,0.409253,2.0,0.0,0.000000,0.0,0.000000,0.0,0.0
202426,4720,44,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0


#### Time since last and first purchase of item in shop

In [98]:
with_last_purchase_month = df.groupby(["item_id", "shop_id"]).agg({"date_block_num":"max"}).reset_index().rename(columns={"date_block_num":"month_last_purchase_of_item_in_shop"})
with_first_purchase_month = df.groupby(["item_id", "shop_id"]).agg({"date_block_num":"min"}).reset_index().rename(columns={"date_block_num":"month_first_purchase_of_item_in_shop"})
print(len(with_last_purchase_month))
print(len(with_first_purchase_month))

391352
391352


In [99]:
print(len(features_items_and_shops))
features_items_and_shops = pd.merge(features_items_and_shops, with_last_purchase_month, on=["shop_id", "item_id"], how="left")
features_items_and_shops = pd.merge(features_items_and_shops, with_first_purchase_month, on=["shop_id", "item_id"], how="left")
print(len(features_items_and_shops))
features_items_and_shops.sample(10)

214200
214200


,item_id,shop_id,purchases_shop_last_month,purchases_item_in_shop_month_2,purchases_item_in_shop_month_3,hist_sales_mean_item_by_shop,hist_sales_sum_item_by_shop,hist_sales_std_item_by_shop,hist_sales_max_item_by_shop,hist_sales_min_item_by_shop,ten_month_mean_item_by_shop,ten_month_sales_sum_item_by_shop,ten_month_std_item_by_shop,ten_month_max_item_by_shop,ten_month_min_item_by_shop,month_last_purchase_of_item_in_shop,month_first_purchase_of_item_in_shop
7447,9753,4,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN
208490,5515,39,0.0,0.0,0.0,0.034483,1.0,0.185695,1.0,0.0,0.090909,1.0,0.301511,1.0,0.0,22.0,22.0
89711,15900,18,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN
137161,15574,48,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN
36382,14627,12,0.0,0.0,0.0,0.034483,1.0,0.185695,1.0,0.0,0.090909,1.0,0.301511,1.0,0.0,23.0,23.0
100306,6957,19,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN
122474,7133,52,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN
26153,18466,7,0.0,0.0,0.0,0.413793,12.0,0.501230,1.0,0.0,0.363636,4.0,0.504525,1.0,0.0,24.0,0.0
30167,4135,7,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN
123476,6532,52,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN


#### Number of months in there was a sell of item in shop

In [100]:
total_months_with_sells = df.groupby(["item_id", "shop_id"]).agg({"date_block_num":"nunique"}).reset_index().rename(columns={"date_block_num":"total_months_with_sells_by_item_and_shop"})
total_months_with_sells.sample(7)

,item_id,shop_id,total_months_with_sells_by_item_and_shop
341650,19130,4,1
257844,14853,25,3
2537,313,54,7
342902,19177,58,4
30303,2333,46,3
158334,9639,23,1
223901,13013,16,2


In [101]:
print(len(features_items_and_shops))
features_items_and_shops = pd.merge(features_items_and_shops, total_months_with_sells, on=["item_id", "shop_id"], how="left")
print(len(features_items_and_shops))
features_items_and_shops.sample(7)

214200
214200


,item_id,shop_id,purchases_shop_last_month,purchases_item_in_shop_month_2,purchases_item_in_shop_month_3,hist_sales_mean_item_by_shop,hist_sales_sum_item_by_shop,hist_sales_std_item_by_shop,hist_sales_max_item_by_shop,hist_sales_min_item_by_shop,ten_month_mean_item_by_shop,ten_month_sales_sum_item_by_shop,ten_month_std_item_by_shop,ten_month_max_item_by_shop,ten_month_min_item_by_shop,month_last_purchase_of_item_in_shop,month_first_purchase_of_item_in_shop,total_months_with_sells_by_item_and_shop
112300,4996,49,1.0,0.0,1.0,0.103448,3.0,0.309934,1.0,0.0,0.272727,3.0,0.467099,1.0,0.0,28.0,25.0,3.0
87850,2039,18,2.0,2.0,0.0,0.413793,12.0,0.945563,3.0,0.0,1.090909,12.0,1.300350,3.0,0.0,28.0,21.0,5.0
141999,13403,57,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN
139399,6496,57,1.0,0.0,1.0,0.620690,18.0,0.941647,4.0,0.0,0.272727,3.0,0.467099,1.0,0.0,28.0,6.0,12.0
208197,13298,39,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN
210468,20739,45,0.0,0.0,0.0,0.241379,7.0,0.689470,3.0,0.0,0.636364,7.0,1.026911,3.0,0.0,23.0,19.0,4.0
197390,3230,41,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,NaN,NaN,NaN


In [102]:
len(features_items_and_shops)

214200

In [103]:
features_items_and_shops.to_csv("generated/" + features_folder_name + "features_items_and_shop.csv", index=False)

### Of items (item_id)

In [18]:
items_features = df_aspects[["item_id"]].drop_duplicates(subset=["item_id"])
print(len(items_features))
items_features.head()

5100


,item_id
30,5037
61,5233
92,4872
123,5002
154,5814


#### Max, mean, std, and min item price

In [105]:
df_prices = df_aux.copy()

In [106]:
with_price_features = df_prices.groupby("item_id").agg({"item_price":["max", "mean", "min", "std"]}).reset_index()
with_price_features.columns = ['_'.join(col).strip() for col in with_price_features.columns.values]
with_price_features["diff_price_max_min"] = with_price_features["item_price_max"] - with_price_features["item_price_min"]
with_price_features.rename(columns={"item_id_":"item_id"}, inplace=True)

print(len(items_features))
items_features = pd.merge(items_features, with_price_features, on="item_id", how="left")
print(len(items_features))

items_features.sample(7)

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min
4577,8116,1699.0,1699.000000,1699.00,0.000000,0.00
2560,9366,499.0,488.398500,175.94,53.077735,323.06
3573,2428,NaN,NaN,NaN,NaN,NaN
706,7452,549.0,467.075351,294.29,42.204683,254.71
4583,22054,2199.0,2199.000000,2199.00,0.000000,0.00
2018,11404,399.0,396.946569,265.00,15.404053,134.00
2283,11268,149.0,102.172619,77.00,13.299491,72.00


In [107]:
len(items_features)

5100

#### Total shops that sell specific item

In [108]:
with_different_shops = df_prices.groupby("item_id").agg({"shop_id":"nunique"}).reset_index().rename(columns={"shop_id":"total_shops_item_is_sell"})
print(len(items_features))
items_features = pd.merge(items_features, with_different_shops, on="item_id", how="left")
items_features["total_shops_item_is_sell"].fillna(0)
print(len(items_features))

items_features.sample(10)

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell
901,5044,2999.0,2855.730769,2499.00,227.309249,500.00,25.0
961,5041,3999.0,3829.722935,3062.65,236.618784,936.35,41.0
5024,4203,399.0,305.823529,300.00,24.011027,99.00,1.0
545,5026,299.0,297.006098,190.00,14.651576,109.00,41.0
4007,7572,NaN,NaN,NaN,NaN,NaN,NaN
2197,22007,149.0,149.000000,149.00,0.000000,0.00,4.0
2447,8400,399.0,260.778497,63.00,130.362331,336.00,43.0
3980,7570,1890.0,1676.333333,1249.00,370.081523,641.00,2.0
2219,15784,299.0,248.387593,89.48,70.998738,209.52,48.0
3790,19979,3199.0,3199.000000,3199.00,0.000000,0.00,2.0


#### Mean sales of item historically by month

In [109]:
items_sells_historically = df_ts.groupby(["item_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_items_sells = items_sells_historically.groupby("item_id").agg({"item_cnt_month":"mean"}).reset_index().rename(columns={"item_cnt_month":"sales_item_historically_mean"})
print(len(items_features))
items_features = pd.merge(items_features, mean_items_sells, on="item_id", how="left")
print(len(items_features))
items_features.sample(10)

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean
351,18360,199.0,193.954785,102.18,20.657987,96.82,43.0,8.275862
3733,13580,1799.0,1799.000000,1799.00,0.000000,0.00,2.0,0.000000
4834,325,249.0,242.662202,75.00,31.291093,174.00,27.0,4.000000
3335,10194,499.0,499.000000,499.00,0.000000,0.00,1.0,3.310345
822,3442,1799.0,1212.195205,899.00,393.264923,900.00,38.0,0.000000
182,16549,229.0,198.718548,110.00,16.223309,119.00,44.0,9.931034
4117,15447,1299.0,1138.272347,567.00,170.730316,732.00,36.0,5.931034
3422,21803,175.0,163.695652,149.00,13.178615,26.00,1.0,0.724138
2829,17321,549.0,405.427083,148.00,157.472721,401.00,39.0,2.034483
4512,6867,1549.0,1549.000000,1549.00,0.000000,0.00,6.0,0.000000


#### Mean of item sales last ten months

In [110]:
items_sells_historically_10 = df_ts[df_ts.date_block_num >= max_train_month - 10].groupby(["item_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
mean_items_sells_10 = items_sells_historically_10.groupby("item_id").agg({"item_cnt_month":"mean"}).reset_index().rename(columns={"item_cnt_month":"sales_item_mean_ten"})
print(len(items_features))
items_features = pd.merge(items_features, mean_items_sells_10, on="item_id", how="left")
print(len(items_features))
items_features.sample(10)

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten
1866,8569,399.0,396.308354,242.03,17.728972,156.97,42.0,11.310345,29.818182
4752,14220,299.0,288.780000,167.00,35.391916,132.00,19.0,3.517241,2.272727
3665,14152,3199.0,3193.925714,3000.00,28.950101,199.00,22.0,1.758621,4.636364
1763,10074,249.0,249.000000,249.00,0.000000,0.00,6.0,0.241379,0.636364
3101,10659,999.0,778.152420,412.00,150.348799,587.00,51.0,57.275862,77.454545
4191,13226,659.0,659.000000,659.00,0.000000,0.00,10.0,0.000000,0.000000
2214,9797,229.0,163.337838,72.00,50.146016,157.00,42.0,5.655172,8.181818
1210,5272,1199.0,327.207278,143.56,121.194853,1055.44,54.0,160.241379,106.909091
5023,4145,990.0,990.000000,990.00,0.000000,0.00,1.0,0.000000,0.000000
2796,16866,499.0,156.893617,70.00,101.704225,429.00,19.0,0.517241,0.090909


#### Number of months where item has sales

In [111]:
df.head()

,item_id,item_category_id,item_category_type,item_category_subtype,date_block_num,shop_id,item_price,item_cnt_day,shop_city
0,0,40,Кино,DVD,20,54,58.0,1.0,Химки
1,2,40,Кино,DVD,19,54,58.0,1.0,Химки
2,2,40,Кино,DVD,22,54,58.0,1.0,Химки
3,3,40,Кино,DVD,18,54,100.0,1.0,Химки
4,3,40,Кино,DVD,19,54,58.0,1.0,Химки


In [112]:
months_of_sales_by_item = df.groupby("item_id").agg({"date_block_num":"nunique"}).reset_index().rename(columns={"date_block_num":"months_item_has_sales"})
print(len(items_features))
items_features = pd.merge(items_features, months_of_sales_by_item, on="item_id", how="left")
print(len(items_features))
items_features.sample(10)

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales
4248,13245,549.0,549.000000,549.00,0.000000,0.00,2.0,0.000000,0.000000,NaN
3087,14423,599.0,560.440631,354.50,45.520475,244.50,47.0,10.000000,26.363636,11.0
2114,11188,449.0,416.000709,252.74,50.471341,196.26,32.0,1.137931,3.000000,1.0
2453,18887,149.0,114.666667,78.00,35.362504,71.00,19.0,0.896552,2.363636,7.0
3688,21001,1899.0,1899.000000,1899.00,0.000000,0.00,2.0,0.000000,0.000000,NaN
1928,17887,399.0,392.587826,256.74,27.593705,142.26,31.0,1.965517,5.181818,3.0
18,6928,2599.0,1715.005563,1078.31,443.024202,1520.69,47.0,3.931034,6.818182,9.0
68,7005,2599.0,2070.269091,999.00,522.815913,1600.00,50.0,23.551724,22.272727,16.0
325,12974,249.0,245.661282,173.00,15.495298,76.00,45.0,13.000000,11.000000,29.0
817,3352,2999.0,2928.630393,2292.35,159.927302,706.65,43.0,0.000000,0.000000,NaN


#### Total times item is sell in one unit, two units, etc..

In [113]:
df.head()

,item_id,item_category_id,item_category_type,item_category_subtype,date_block_num,shop_id,item_price,item_cnt_day,shop_city
0,0,40,Кино,DVD,20,54,58.0,1.0,Химки
1,2,40,Кино,DVD,19,54,58.0,1.0,Химки
2,2,40,Кино,DVD,22,54,58.0,1.0,Химки
3,3,40,Кино,DVD,18,54,100.0,1.0,Химки
4,3,40,Кино,DVD,19,54,58.0,1.0,Химки


In [114]:
count_items = df["item_cnt_day"].value_counts()

In [115]:
max_units = 5
max_units

5

In [116]:
print(len(items_features))
df_aux_1 = df.copy()
for x in range(1, max_units + 1):
    df_aux_1["total_sales_with_" + str(x) + "_units_last_month"] = 1
    counting = df_aux_1[(df_aux_1.date_block_num == max_train_month) & (df_aux_1.item_cnt_day == x)].groupby("item_id").agg({"total_sales_with_" + str(x) + "_units_last_month":"sum"}).reset_index()
    counting["total_sales_with_" + str(x) + "_units_last_month"].fillna(0, inplace=True)
    items_features = pd.merge(items_features, counting, on="item_id", how="left")
print(len(items_features))
items_features.sample(10)

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month
1146,13159,299.0,221.717693,104.32,65.604001,194.68,52.0,49.413793,116.000000,13.0,58.0,2.0,NaN,NaN,NaN
5000,536,1700.0,1700.000000,1700.00,0.000000,0.00,1.0,0.931034,1.727273,8.0,NaN,NaN,NaN,NaN,NaN
2847,17204,699.0,312.907143,159.47,150.237867,539.53,20.0,0.724138,1.545455,9.0,4.0,NaN,NaN,NaN,NaN
43,3026,2599.0,2298.513347,1299.00,525.310097,1300.00,48.0,19.551724,51.545455,7.0,60.0,3.0,NaN,NaN,NaN
4643,5380,3799.0,3251.783576,1882.00,395.083392,1917.00,49.0,57.103448,51.272727,29.0,37.0,1.0,NaN,NaN,NaN
571,5551,399.0,394.874924,245.25,22.621995,153.75,41.0,9.448276,24.909091,2.0,84.0,4.0,NaN,NaN,NaN
386,12970,229.0,199.760140,91.11,19.556124,137.89,51.0,15.724138,15.363636,29.0,14.0,NaN,NaN,NaN,NaN
2799,11631,499.0,444.566667,283.00,91.881364,216.00,19.0,0.551724,0.272727,11.0,NaN,NaN,NaN,NaN,NaN
4657,4956,699.0,678.090909,469.00,67.145534,230.00,18.0,0.482759,1.272727,6.0,2.0,NaN,NaN,NaN,NaN
1635,2425,499.0,499.000000,499.00,0.000000,0.00,4.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN


#### Rolling mean, max, min, std of sales across every month

In [117]:
items_sells_historically = df_ts[df_ts.item_id.isin(items_features.item_id)].groupby(["item_id", "date_block_num"]).agg({"item_cnt_month":"sum"}).reset_index()
grouped = items_sells_historically.sort_values(by="date_block_num").groupby("item_id").agg({"item_cnt_month":[rolling_mean, rolling_std, rolling_max, rolling_min, diffmean]}).reset_index()
grouped.columns = ['_'.join(col).strip() + "_item" for col in grouped.columns.values]
grouped.rename(columns={"item_id__item":"item_id"}, inplace=True)
grouped.head()

,item_id,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item
0,30,42.604938,67.193180,272.000000,4.333333,0.238095
1,31,30.061728,47.451103,196.333333,9.000000,0.222222
2,32,50.975309,34.596938,166.000000,18.000000,-7.083333
3,33,21.246914,7.821509,35.000000,10.333333,-1.650000
4,38,1.259259,1.970581,6.000000,0.000000,1.500000


In [118]:
print(len(items_features))
items_features = pd.merge(items_features, grouped, on="item_id", how="left")
print(len(items_features))
items_features.head()

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item
0,5037,2599.0,1926.828388,749.00,655.166962,1850.00,50.0,25.103448,66.181818,9.0,73.0,7.0,NaN,NaN,NaN,24.629630,39.208661,111.0,0.0,3.125000
1,5233,1199.0,800.777500,599.00,281.878686,600.00,43.0,2.586207,6.818182,2.0,61.0,5.0,NaN,NaN,NaN,1.024691,4.818878,25.0,0.0,33.500000
2,4872,1499.0,1001.614545,524.44,190.123528,974.56,51.0,102.275862,216.090909,13.0,113.0,5.0,NaN,NaN,NaN,105.962963,120.193659,289.0,0.0,8.692308
3,5002,2999.0,2999.000000,2999.00,0.000000,0.00,16.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,NaN
4,5814,1199.0,1088.667036,599.00,228.975514,600.00,50.0,17.068966,45.000000,10.0,42.0,3.0,1.0,NaN,NaN,16.962963,23.826354,63.0,0.0,4.700000


#### Numbers of items sold shifted n different months

In [119]:
months = [month_to_predict - i for i in [1,2,3,6]]
months

[28, 27, 26, 23]

In [21]:
df_ts[(df_ts.item_id == 213) & (df_ts.date_block_num == 28)]["item_cnt_month"].mean()

0.07142857142857142

In [31]:
a = list(df_ts[(df_ts.item_id == 213) & (df_ts.date_block_num == 28)]["shop_id"].value_counts().index)

In [32]:
b = [59, 58, 28, 27, 26, 25, 24, 22, 21, 19, 18, 16, 15, 14, 12, 11, 7, 6, 5, 4, 3, 29, 31, 34, 48, 57, 56, 55, 53, 52, 51, 50, 49, 47, 35, 46, 45, 44, 42, 41, 39, 38, 37, 2]

In [33]:
[x for x in a if x not in b]

[10, 36]

In [34]:
[x for x in b if x not in a]

[27, 11, 29, 51]

In [120]:
print(len(items_features))
for month in months:
    items_sold_in_month = df_ts[df_ts["date_block_num"] == month].groupby("item_id").agg({"item_cnt_month":["mean", "sum", "max", "std"]}).reset_index()
    items_sold_in_month.columns = ['_'.join(col).strip() for col in items_sold_in_month.columns.values]
    
    items_sold_in_month.rename(columns={
        "item_id_":"item_id",
        "item_cnt_month_sum":"item_purchases_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_mean":"item_purchases_mean_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_max":"item_purchases_max_shifted_" + str(month_to_predict - month) + "_months",
        "item_cnt_month_std":"item_purchases_std_shifted_" + str(month_to_predict - month) + "_months",
    },
    inplace=True)
    items_features = pd.merge(items_features, items_sold_in_month, on="item_id", how="left")
print(len(items_features))
items_features.head()

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item,item_purchases_mean_shifted_1_months,item_purchases_shifted_1_months,item_purchases_max_shifted_1_months,item_purchases_std_shifted_1_months,item_purchases_mean_shifted_2_months,item_purchases_shifted_2_months,item_purchases_max_shifted_2_months,item_purchases_std_shifted_2_months,item_purchases_mean_shifted_3_months,item_purchases_shifted_3_months,item_purchases_max_shifted_3_months,item_purchases_std_shifted_3_months,item_purchases_mean_shifted_6_months,item_purchases_shifted_6_months,item_purchases_max_shifted_6_months,item_purchases_std_shifted_6_months
0,5037,2599.0,1926.828388,749.00,655.166962,1850.00,50.0,25.103448,66.181818,9.0,73.0,7.0,NaN,NaN,NaN,24.629630,39.208661,111.0,0.0,3.125000,1.952381,82.0,6.0,1.607185,0.595238,25.0,2.0,0.700506,0.666667,28.0,3.0,0.845841,4.738095,199.0,14.0,3.541114
1,5233,1199.0,800.777500,599.00,281.878686,600.00,43.0,2.586207,6.818182,2.0,61.0,5.0,NaN,NaN,NaN,1.024691,4.818878,25.0,0.0,33.500000,1.595238,67.0,6.0,1.638850,0.190476,8.0,2.0,0.454683,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,4872,1499.0,1001.614545,524.44,190.123528,974.56,51.0,102.275862,216.090909,13.0,113.0,5.0,NaN,NaN,NaN,105.962963,120.193659,289.0,0.0,8.692308,2.690476,113.0,12.0,2.363215,2.119048,89.0,9.0,2.177517,4.047619,170.0,17.0,4.125781,8.761905,368.0,20.0,6.075980
3,5002,2999.0,2999.000000,2999.00,0.000000,0.00,16.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,5814,1199.0,1088.667036,599.00,228.975514,600.00,50.0,17.068966,45.000000,10.0,42.0,3.0,1.0,NaN,NaN,16.962963,23.826354,63.0,0.0,4.700000,1.119048,47.0,5.0,1.233721,0.404762,17.0,2.0,0.700506,1.071429,45.0,6.0,1.276291,1.333333,56.0,7.0,1.508782


In [121]:
items_features[(items_features.item_id == 4719)][:1]#["item_cnt_month"].mean()

,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item,item_purchases_mean_shifted_1_months,item_purchases_shifted_1_months,item_purchases_max_shifted_1_months,item_purchases_std_shifted_1_months,item_purchases_mean_shifted_2_months,item_purchases_shifted_2_months,item_purchases_max_shifted_2_months,item_purchases_std_shifted_2_months,item_purchases_mean_shifted_3_months,item_purchases_shifted_3_months,item_purchases_max_shifted_3_months,item_purchases_std_shifted_3_months,item_purchases_mean_shifted_6_months,item_purchases_shifted_6_months,item_purchases_max_shifted_6_months,item_purchases_std_shifted_6_months
1101,4719,999.0,987.573623,707.92,45.345944,291.08,42.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Delta price lag

In [122]:
months = [month_to_predict - i for i in [1,2,3,6]]
months

[28, 27, 26, 23]

In [123]:
print(len(items_features))
for month in months:
    item_price_mean = df[df["date_block_num"] == month].groupby("item_id").agg({"item_price":"mean"}).reset_index()
    item_price_mean.rename(columns={
        "item_price":"item_price_mean_shifted_" + str(month_to_predict - month) + "_months",
    }, inplace=True)
    items_features = pd.merge(items_features, item_price_mean, on="item_id", how="left")
print(len(items_features))
items_features.head()

5100
5100


,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item,item_purchases_mean_shifted_1_months,item_purchases_shifted_1_months,item_purchases_max_shifted_1_months,item_purchases_std_shifted_1_months,item_purchases_mean_shifted_2_months,item_purchases_shifted_2_months,item_purchases_max_shifted_2_months,item_purchases_std_shifted_2_months,item_purchases_mean_shifted_3_months,item_purchases_shifted_3_months,item_purchases_max_shifted_3_months,item_purchases_std_shifted_3_months,item_purchases_mean_shifted_6_months,item_purchases_shifted_6_months,item_purchases_max_shifted_6_months,item_purchases_std_shifted_6_months,item_price_mean_shifted_1_months,item_price_mean_shifted_2_months,item_price_mean_shifted_3_months,item_price_mean_shifted_6_months
0,5037,2599.0,1926.828388,749.00,655.166962,1850.00,50.0,25.103448,66.181818,9.0,73.0,7.0,NaN,NaN,NaN,24.629630,39.208661,111.0,0.0,3.125000,1.952381,82.0,6.0,1.607185,0.595238,25.0,2.0,0.700506,0.666667,28.0,3.0,0.845841,4.738095,199.0,14.0,3.541114,1542.868750,2599.000000,2599.000000,2073.978495
1,5233,1199.0,800.777500,599.00,281.878686,600.00,43.0,2.586207,6.818182,2.0,61.0,5.0,NaN,NaN,NaN,1.024691,4.818878,25.0,0.0,33.500000,1.595238,67.0,6.0,1.638850,0.190476,8.0,2.0,0.454683,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,790.034091,1199.000000,NaN,NaN
2,4872,1499.0,1001.614545,524.44,190.123528,974.56,51.0,102.275862,216.090909,13.0,113.0,5.0,NaN,NaN,NaN,105.962963,120.193659,289.0,0.0,8.692308,2.690476,113.0,12.0,2.363215,2.119048,89.0,9.0,2.177517,4.047619,170.0,17.0,4.125781,8.761905,368.0,20.0,6.075980,1192.157311,1196.084043,1192.468675,893.628897
3,5002,2999.0,2999.000000,2999.00,0.000000,0.00,16.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,NaN,NaN,NaN,NaN
4,5814,1199.0,1088.667036,599.00,228.975514,600.00,50.0,17.068966,45.000000,10.0,42.0,3.0,1.0,NaN,NaN,16.962963,23.826354,63.0,0.0,4.700000,1.119048,47.0,5.0,1.233721,0.404762,17.0,2.0,0.700506,1.071429,45.0,6.0,1.276291,1.333333,56.0,7.0,1.508782,820.085870,1199.000000,1199.000000,1187.276290


In [124]:
for month in months:
    items_features["delta_price_lag_" + str(month_to_predict - month)] = \
        (items_features["item_price_mean_shifted_" + str(month_to_predict - month)+"_months"] - items_features["item_price_mean"]) / items_features["item_price_mean"]
    
items_features.head()

,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item,item_purchases_mean_shifted_1_months,item_purchases_shifted_1_months,item_purchases_max_shifted_1_months,item_purchases_std_shifted_1_months,item_purchases_mean_shifted_2_months,item_purchases_shifted_2_months,item_purchases_max_shifted_2_months,item_purchases_std_shifted_2_months,item_purchases_mean_shifted_3_months,item_purchases_shifted_3_months,item_purchases_max_shifted_3_months,item_purchases_std_shifted_3_months,item_purchases_mean_shifted_6_months,item_purchases_shifted_6_months,item_purchases_max_shifted_6_months,item_purchases_std_shifted_6_months,item_price_mean_shifted_1_months,item_price_mean_shifted_2_months,item_price_mean_shifted_3_months,item_price_mean_shifted_6_months,delta_price_lag_1,delta_price_lag_2,delta_price_lag_3,delta_price_lag_6
0,5037,2599.0,1926.828388,749.00,655.166962,1850.00,50.0,25.103448,66.181818,9.0,73.0,7.0,NaN,NaN,NaN,24.629630,39.208661,111.0,0.0,3.125000,1.952381,82.0,6.0,1.607185,0.595238,25.0,2.0,0.700506,0.666667,28.0,3.0,0.845841,4.738095,199.0,14.0,3.541114,1542.868750,2599.000000,2599.000000,2073.978495,-0.199270,0.348849,0.348849,0.076369
1,5233,1199.0,800.777500,599.00,281.878686,600.00,43.0,2.586207,6.818182,2.0,61.0,5.0,NaN,NaN,NaN,1.024691,4.818878,25.0,0.0,33.500000,1.595238,67.0,6.0,1.638850,0.190476,8.0,2.0,0.454683,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,790.034091,1199.000000,NaN,NaN,-0.013416,0.497295,NaN,NaN
2,4872,1499.0,1001.614545,524.44,190.123528,974.56,51.0,102.275862,216.090909,13.0,113.0,5.0,NaN,NaN,NaN,105.962963,120.193659,289.0,0.0,8.692308,2.690476,113.0,12.0,2.363215,2.119048,89.0,9.0,2.177517,4.047619,170.0,17.0,4.125781,8.761905,368.0,20.0,6.075980,1192.157311,1196.084043,1192.468675,893.628897,0.190236,0.194156,0.190546,-0.107812
3,5002,2999.0,2999.000000,2999.00,0.000000,0.00,16.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5814,1199.0,1088.667036,599.00,228.975514,600.00,50.0,17.068966,45.000000,10.0,42.0,3.0,1.0,NaN,NaN,16.962963,23.826354,63.0,0.0,4.700000,1.119048,47.0,5.0,1.233721,0.404762,17.0,2.0,0.700506,1.071429,45.0,6.0,1.276291,1.333333,56.0,7.0,1.508782,820.085870,1199.000000,1199.000000,1187.276290,-0.246706,0.101347,0.101347,0.090578


In [125]:
months = [month_to_predict - i for i in [1,2,3,6]]
months

def select_trend(row):
    for month in months:
        if row['delta_price_lag_'+str(str(month_to_predict - month))]:
            return row['delta_price_lag_'+str(str(month_to_predict - month))]
    return 0

In [126]:
items_features['delta_price_lag'] = items_features.apply(select_trend, axis=1)
items_features.head()

,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item,item_purchases_mean_shifted_1_months,item_purchases_shifted_1_months,item_purchases_max_shifted_1_months,item_purchases_std_shifted_1_months,item_purchases_mean_shifted_2_months,item_purchases_shifted_2_months,item_purchases_max_shifted_2_months,item_purchases_std_shifted_2_months,item_purchases_mean_shifted_3_months,item_purchases_shifted_3_months,item_purchases_max_shifted_3_months,item_purchases_std_shifted_3_months,item_purchases_mean_shifted_6_months,item_purchases_shifted_6_months,item_purchases_max_shifted_6_months,item_purchases_std_shifted_6_months,item_price_mean_shifted_1_months,item_price_mean_shifted_2_months,item_price_mean_shifted_3_months,item_price_mean_shifted_6_months,delta_price_lag_1,delta_price_lag_2,delta_price_lag_3,delta_price_lag_6,delta_price_lag
0,5037,2599.0,1926.828388,749.00,655.166962,1850.00,50.0,25.103448,66.181818,9.0,73.0,7.0,NaN,NaN,NaN,24.629630,39.208661,111.0,0.0,3.125000,1.952381,82.0,6.0,1.607185,0.595238,25.0,2.0,0.700506,0.666667,28.0,3.0,0.845841,4.738095,199.0,14.0,3.541114,1542.868750,2599.000000,2599.000000,2073.978495,-0.199270,0.348849,0.348849,0.076369,-0.199270
1,5233,1199.0,800.777500,599.00,281.878686,600.00,43.0,2.586207,6.818182,2.0,61.0,5.0,NaN,NaN,NaN,1.024691,4.818878,25.0,0.0,33.500000,1.595238,67.0,6.0,1.638850,0.190476,8.0,2.0,0.454683,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,790.034091,1199.000000,NaN,NaN,-0.013416,0.497295,NaN,NaN,-0.013416
2,4872,1499.0,1001.614545,524.44,190.123528,974.56,51.0,102.275862,216.090909,13.0,113.0,5.0,NaN,NaN,NaN,105.962963,120.193659,289.0,0.0,8.692308,2.690476,113.0,12.0,2.363215,2.119048,89.0,9.0,2.177517,4.047619,170.0,17.0,4.125781,8.761905,368.0,20.0,6.075980,1192.157311,1196.084043,1192.468675,893.628897,0.190236,0.194156,0.190546,-0.107812,0.190236
3,5002,2999.0,2999.000000,2999.00,0.000000,0.00,16.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.0,0.0,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5814,1199.0,1088.667036,599.00,228.975514,600.00,50.0,17.068966,45.000000,10.0,42.0,3.0,1.0,NaN,NaN,16.962963,23.826354,63.0,0.0,4.700000,1.119048,47.0,5.0,1.233721,0.404762,17.0,2.0,0.700506,1.071429,45.0,6.0,1.276291,1.333333,56.0,7.0,1.508782,820.085870,1199.000000,1199.000000,1187.276290,-0.246706,0.101347,0.101347,0.090578,-0.246706


In [127]:
features_to_drop = ["delta_price_lag_" + str(i) for i in [1,2,3,6]]
features_to_drop.extend(["item_price_mean_shifted_" +str(i)+"_months" for i in [1,2,3,6]])
features_to_drop

['delta_price_lag_1',
 'delta_price_lag_2',
 'delta_price_lag_3',
 'delta_price_lag_6',
 'item_price_mean_shifted_1_months',
 'item_price_mean_shifted_2_months',
 'item_price_mean_shifted_3_months',
 'item_price_mean_shifted_6_months']

In [128]:
items_features.drop(columns=features_to_drop, inplace=True)
items_features.sample(5)

,item_id,item_price_max,item_price_mean,item_price_min,item_price_std,diff_price_max_min,total_shops_item_is_sell,sales_item_historically_mean,sales_item_mean_ten,months_item_has_sales,total_sales_with_1_units_last_month,total_sales_with_2_units_last_month,total_sales_with_3_units_last_month,total_sales_with_4_units_last_month,total_sales_with_5_units_last_month,item_cnt_month_rolling_mean_item,item_cnt_month_rolling_std_item,item_cnt_month_rolling_max_item,item_cnt_month_rolling_min_item,item_cnt_month_diffmean_item,item_purchases_mean_shifted_1_months,item_purchases_shifted_1_months,item_purchases_max_shifted_1_months,item_purchases_std_shifted_1_months,item_purchases_mean_shifted_2_months,item_purchases_shifted_2_months,item_purchases_max_shifted_2_months,item_purchases_std_shifted_2_months,item_purchases_mean_shifted_3_months,item_purchases_shifted_3_months,item_purchases_max_shifted_3_months,item_purchases_std_shifted_3_months,item_purchases_mean_shifted_6_months,item_purchases_shifted_6_months,item_purchases_max_shifted_6_months,item_purchases_std_shifted_6_months,delta_price_lag
5018,4187,1990.0,1990.000000,1990.0,0.000000,0.0,1.0,0.275862,0.727273,6.0,NaN,NaN,NaN,NaN,NaN,0.271605,0.453260,1.333333,0.0,1.000,0.000000,0.0,0.0,0.000000,0.047619,2.0,2.0,0.308607,0.023810,1.0,1.0,0.154303,0.023810,1.0,1.0,0.154303,NaN
3809,494,380.0,379.586957,323.0,4.852162,57.0,17.0,3.724138,6.454545,15.0,3.0,NaN,NaN,NaN,NaN,3.814815,3.942247,11.000000,0.0,0.250,0.071429,3.0,2.0,0.341650,0.214286,9.0,5.0,0.842057,0.047619,2.0,2.0,0.308607,0.214286,9.0,7.0,1.116084,0.001088
448,1241,299.0,296.699416,170.0,16.417832,129.0,45.0,6.413793,6.363636,29.0,10.0,NaN,NaN,NaN,NaN,6.456790,1.849343,10.333333,3.0,0.375,0.190476,8.0,2.0,0.454683,0.095238,4.0,2.0,0.370203,0.142857,6.0,2.0,0.417392,0.166667,7.0,2.0,0.437101,0.007754
1115,6742,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.0,NaN,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,NaN
2082,19023,399.0,319.894254,148.0,99.185983,251.0,36.0,3.620690,5.363636,13.0,NaN,NaN,NaN,NaN,NaN,3.851852,4.702548,15.333333,0.0,0.375,0.000000,0.0,0.0,0.000000,0.071429,3.0,1.0,0.260661,0.071429,3.0,1.0,0.260661,0.166667,7.0,2.0,0.437101,NaN


In [129]:
print(len(items_features))

5100


In [130]:
items_features.to_csv("generated/" + features_folder_name + "features_items.csv", index=False)

### Of months

#### Stats last months

In [131]:
df_aux.head()

,item_name,item_id,item_category_id,item_category_name,item_category_type,item_category_subtype,date,date_block_num,shop_id,item_price,item_cnt_day,shop_name,shop_city
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,Кино - DVD,Кино,DVD,01.09.2014,20,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
1,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,24.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,Кино - DVD,Кино,DVD,12.11.2014,22,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,05.07.2014,18,54,100.0,1.0,"Химки ТЦ ""Мега""",Химки
4,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,Кино - DVD,Кино,DVD,26.08.2014,19,54,58.0,1.0,"Химки ТЦ ""Мега""",Химки


In [132]:
group = df_aux.groupby(["item_id", "shop_id", "date_block_num"]).agg({"item_cnt_day":"sum"}).reset_index().rename(columns={"item_cnt_day":"item_cnt_month"})
print(group.date_block_num.max())
month_means = group.groupby("date_block_num").agg({"item_cnt_month":"mean"}).reset_index().rename(columns={"item_cnt_month":"month_mean"})
# add one to merge with labeled month
month_means["date_block_num"] += 1
month_means.head()

33


,date_block_num,month_mean
0,1,2.079574
1,2,2.137149
2,3,2.299920
3,4,1.961821
4,5,2.007092


In [133]:
month_means.to_csv("generated/features_months.csv", index=False)

### Text features from descriptions of items

In [265]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [266]:
items = pd.read_csv("data/items.csv")
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [267]:
test_items_ids = list(pd.read_csv("data/test.csv")["item_id"])
items = items[items["item_id"].isin(test_items_ids)]

In [268]:
items.head()

,item_name,item_id,item_category_id
30,007: КООРДИНАТЫ «СКАЙФОЛЛ»,30,40
31,007: КООРДИНАТЫ «СКАЙФОЛЛ» (BD),31,37
32,1+1,32,40
33,1+1 (BD),33,37
38,10 САМЫХ ПОПУЛЯРНЫХ КИНОКОМЕДИЙ ХХ ВЕКА 10DVD...,38,41


#### TF-IDF items description with 50 more frequent words

In [269]:
corpus = items["item_name"]
vectorizer = TfidfVectorizer(max_features=50)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.shape)
matrix = X.toarray()
matrix

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
tf_idf["item_id"] = items["item_id"]

In [ ]:
tf_idf.sample(10)

In [ ]:
tf_idf.to_csv("generated/generals/features_items_voc.csv", index=False)

#### Bag of words items descriptions with 50 more frequent n-2grams

In [ ]:
corpus = items["item_name"]
vectorizer = CountVectorizer(max_features=50, ngram_range=(2,2))
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.shape)
matrix = X.toarray()
matrix

In [ ]:
bag_of_words = pd.DataFrame(data=matrix, columns=vectorizer.get_feature_names())
assert(len(bag_of_words) == len(items))

In [ ]:
bag_of_words["item_id"] = list(items["item_id"])

In [ ]:
bag_of_words.head()

In [ ]:
bag_of_words.to_csv("generated/generals/features_items_bow.csv", index=False)

In [ ]:
sum(vectorizer.vocabulary_.values())

In [ ]:
len(items)

### Text features from descriptions of categories

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
categories = pd.read_csv("data/item_categories.csv")
print(len(categories))
categories.head()

In [ ]:
corpus = categories["item_category_name"]
vectorizer = TfidfVectorizer(max_features=10)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
print(X.shape)
matrix = X.toarray()
matrix

In [ ]:
tf_idf = pd.DataFrame(data=matrix, columns=vectorizer.get_feature_names())
assert(len(tf_idf) == len(categories))

In [ ]:
tf_idf["item_category_id"] = categories["item_category_id"]
tf_idf.sample(10)

In [ ]:
tf_idf.to_csv("generated/generals/features_categories_voc.csv", index=False)